In [110]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '..')

from src.CCLE_postp_function import *

from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio.Helper import * 
import gseapy
from taigapy import TaigaClient
from gsheets import Sheets

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


Loading BokehJS ...

In [17]:
samplesetname="20Q3"
prevname="20Q2"
prevversion=14 #TOTAL
prevprevname = "20Q1"
prevprevversion=22 #INTERNAL
virtual_public='public-20q3-3d35'
virtual_dmc='dmc-20q3-033d'
virtual_internal='internal-20q3-00d0'

workspace2="broad-firecloud-ccle/CCLE_DepMap_RNAseq"
workspace4="broad-genomics-delivery/Cancer_Cell_Line_Factory_CCLF_RNAseq"
workspace5="nci-mimoun-bi-org/CCLF_RNA_2_0"

workspace3="broad-genomics-delivery/CCLE_DepMap_RNAseq"
workspace1="broad-genomics-delivery/Getz_IBM_CellLines_RNASeqData"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_RNAseq"
workspace7="terra-broad-cancer-prod/Getz_IBM_CellLines_RNASeqData"

refworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"
source1="ibm"
source2="ccle"
source3="ccle"
source4="cclf"
source5="cclf"
source6="ccle"
source7="ibm"
release = samplesetname

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
privacy_release_url = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

In [3]:
%%R
genome_version <- 'hg38'
release <- '20Q2'
hg38_cyto_band_reference <- '../JKBio/data/hg38_cytoband.gz'

# Generate sample set from new samples

In [4]:
extract_to_change = {'from_arxspan_id': 'participant',
}

In [5]:
wm1 = dm.WorkspaceManager(workspace1)
#wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)
#wm4 = dm.WorkspaceManager(workspace4)
#wm5 = dm.WorkspaceManager(workspace5)
wm6 = dm.WorkspaceManager(workspace6)
wm7 = dm.WorkspaceManager(workspace7)
refwm = dm.WorkspaceManager(refworkspace)

In [6]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame()

In [ ]:
# we will be missing "primary disease","sm_id", "cellosaurus_id", "gender, "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "comments"
#when SMid: match==
samples, pairs, noarxspan = GetNewCellLinesFromWorkspaces(refworkspace, stype='rna', refurl=refsheet_url, wmfroms = [workspace1, workspace3, workspace6, workspace7], sources=[source1, source3,source6, source7], match=['ACH-','CDS-'], participantslicepos=10, accept_unknowntypes=True, extract=extract_to_change, recomputedate=True)

In [ ]:
samples

In [ ]:
# I am trying to remove duplicates from samples without arxspan ids to then look more into them and see if I have to get data for them or if I should just throw them out
toremov=set()
for k, val in noarxspan.iterrows():
    withsamesize = noarxspan[noarxspan["sample_id"] == val["sample_id"]]
    if len(withsamesize) > 1:
        for l, v in withsamesize.iloc[1:].iterrows():
            toremov.add(l)

In [ ]:
for i in toremov:
    noarxspan = noarxspan.drop(i)

In [ ]:
len(noarxspan)

In [ ]:
noarxspan.sample_id = [i.split('NA_O')[0] for i in noarxspan.sample_id]

In [ ]:
noarxspan['ccle_name'] = [''.join(i.split('_')[1:-1]).split('v')[0] for i in noarxspan.sample_id]
noarxspan['readgroup'] = [i.split('_')[0] for i in noarxspan.sample_id]

In [ ]:
noarxspan['ccle_name'].tolist()

In [ ]:
for i, v in noarxspan.iterrows():
    if not gcp.exists(v['cram_or_bam_path']):
        print(v.ccle_name)
        noarxspan = noarxspan.drop(i)

In [ ]:
toupdate = {"gender":[],
"primary_disease":[],
"sm_id":[],
"cellosaurus_id":[],
"age":[],
"primary_site":[],
"subtype":[],
"subsubtype":[],
"origin":[],
"comments":[],
"patient_id":[]}

In [ ]:
refwm.get_bucket_id()

In [ ]:
# If I have a previous samples I can update unknown data directly
index=[]
notfound=[]
for k, val in samples.iterrows():
    dat = ccle_refsamples[ccle_refsamples['arxspan_id']==val['arxspan_id']]
    if len(dat)>0:
        index.append(k)
        for k, v in toupdate.items():
            toupdate[k].append(dat[k].tolist()[0])
    else:
        notfound.append(k)

In [ ]:
len(toupdate['patient_id'])

In [ ]:
# doing so..
for k, v in toupdate.items():
    samples.loc[index,k] =v

In [ ]:
len(samples.loc[notfound].patient_id)

In [ ]:
# for these samples I will need to check and manually add the data in the list 
samples.loc[notfound]

In [ ]:
# found same patient
a = ["ACH-000635","ACH-000717", "ACH-000864", "ACH-001042", "ACH-001547"]
b = ["ACH-002291","ACH-001672"]

In [ ]:
ccle_refsamples[ccle_refsamples.arxspan_id.isin(a)].patient_id

In [ ]:
# if we found we had cell lines from different patients
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.arxspan_id.isin(a)].index,"patient_id"]="PT-B15oIzKN"

In [ ]:
# duplicate ach-id
dup = {"ACH-001620": "ACH-001605",
"ACH-001621": "ACH-001606"}

In [ ]:
ccle_refsamples[ccle_refsamples.arxspan_id=="ACH-001605"]

In [ ]:
samples.loc["CDS-R2h6fu","arxspan_id"]="ACH-001620"

In [ ]:
samples = changeCellLineNameInNewSet(new = samples, ref=ccle_refsamples, datatype="rna", dupdict=dup)

In [ ]:
samples

In [ ]:
notfound.remove("CDS-R2h6fu")

In [ ]:
#rename ccle_name TODO: ask becky what to do
rename = {"PEDS117": "CCLFPEDS0009T"}

In [ ]:
len(notfound)

## getting the addtional data and writing it here in the right order 'as shown above'
- use the stripped_cell_line_name to find the samples on https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE/edit#gid=356471436. 
- Make sure that we don't have duplicate cell lines in there. Otherwise, use the duplicate renaming function
- copy Primary Site, Primary Disease, Subtype, Comments, Disease Sub-subtype, if they exist. (sometimes subtype and subsubtype are the same.. don't use subsubtype then.
- look for the cell line in cellosaurus, you might need to use one of the aliases given in master depmap pv..
- copy  cellosaurus_id gender age info or write 'U' if they don't exist. 'can be a number or {Embryonic, Children, Adult, Fetus, U} 
- check that it does not say this cell line is not a duplicate from another cell line
- check that if it says this cell line is derived/children/father/samepatient from other cell lines, and that if we have any of the other cell lines, that the patient id is changed to be the same one for all (be sure that you are updating everywhere these patient ids are used)

In [ ]:
# TODO: sample info file has different age annotations
toupdate = {"sm_id":[""]*30,
            
"cellosaurus_id":['CVCL_H248',"","CVCL_S505","CVCL_2277","CVCL_1966","CVCL_M147","CVCL_2024","CVCL_A344","CVCL_8354","CVCL_S506","CVCL_M258","CVCL_2581","CVCL_2604","CVCL_4449","CVCL_2610","CVCL_M266","CVCL_4D12","CVCL_3118","","CVCL_T041","CVCL_1852","CVCL_2777","CVCL_B371", "CVCL_3220","CVCL_1854","CVCL_2223","CVCL_2226","CVCL_1903","CVCL_3423","CVCL_1898"],
"gender":['Female',"Male","Male","Male","Male","Male","Female","Male","Female","Male","Male","Male","Male","Female","Male","U","Female","Male","U",'Female',"Male","Male","Male","Female","Male","Female","Female","Male","Female","Male"],
            
"age":[2,'U',"Embryo","Embryo",7,12,49,"Embryo",52,3,56,57,"U","U","U",18,49,71,"U",55,11,22,10,22,57,61,69,37,"U",42],
            
"primary_site":["haematopoietic_and_lymphoid_tissue","skin","Embryonal","Embryonal","haematopoietic_and_lymphoid_tissue","central_nervous_system","pleural_effusion","Embryonal","ascites","central_nervous_system","biliary_tract","pleural_effusion","lymph_node","lymph_node","lymph_node","biliary_tract","uvea","biliary_tract","bone","pleural_effusion","haematopoietic_and_lymphoid_tissue","lung","haematopoietic_and_lymphoid_tissue","Thymus","ascites","upper_aerodigestive_tract","upper_aerodigestive_tract","haematopoietic_and_lymphoid_tissue","breast","haematopoietic_and_lymphoid_tissue"],
            
"primary_disease":['Leukemia',"Skin Cancer","Embryonal Cancer","Embryonal Cancer","Lymphoma","Brain Cancer","Lymphoma","Teratoma","Ovarian Cancer","Brain Cancer","Bile Duct Cancer","Lung Cancer","Skin Cancer","Skin Cancer","Skin Cancer","Bile Duct Cancer","Eye Cancer","Bile Duct Cancer","Bone Cancer","Breast Cancer","Leukemia","Embryonal Cancer","Lymphoma","Thymic Cancer","Lymphoma","Head and Neck Cancer","Head and Neck Cancer","Lymphoma","Breast Cancer","Lymphoma"],
            
"subtype":['AML',"Melanoma","Carcinoma","Carcinoma","B-cell, Non-Hodgkins, Burkitts","Glioblastoma","B-cell, Non-Hodgkins, Burkitts","Testicular","Adenocarcinoma, clear cell","Medulloblastoma","Cholangiocarcinoma","Mesothelioma","Melanoma","Melanoma","Melanoma","Engineered","Uveal Melanoma","Cholangiocarcinoma","Ewings Sarcoma","Breast Ductal Carcinoma","Acute Lymphoblastic Leukemia (ALL), B-cell","Carcinoma","B-cell, Non-Hodgkins, Burkitts","Carcinoma","B-cell","Squamous Cell Carcinoma","Squamous Cell Carcinoma","B-cell","Carcinoma","B-cell"],
            
"subsubtype":["",'acral',"","","b_cell_burkitt","","","","","","","","","","","engineered_bile_duct","","intrahepatic","","","b_cells","","b_cell_burkitt","anaplastic","","","","","ERneg_HER2pos",""],
            
"origin":['']*30,
            
"comments":["Gifted to Fred Hutchinson Cancer Center by Dr. Jeff Taub at Wayne State University; Received from Academic lab (FHCRC and Taub, Wayne State University)","M1610113 (p9); Mutation: NRASG12R; biopsy site - right dorsal heel; Received from Academic lab (Hayward, QIMRB)","cytoreductive thoracotomy specimen of a man with primary retroperitoneal embryonal carcinoma","retro-peritoneal metastasis of a non seminomatous germ cell carcinoma","","Received from Academic lab (Hong, Broad)","","malignant testicular teratoma (intermediate grade)","ovarian clear cell adenocarcinoma","","Human poorly differentiated cholangiocarcinoma cell line established from biliary tract.","","taken from metastatic site: lymph node","taken from metastatic site: lymph node","taken from metastatic site: lymph node","Received from Academic lab (Bardeesy, MGH)/DSMZ","display GNAQ or GNA11 activating mutations","Received from Academic lab (Bardeesy, MGH)/DSMZ","(PCR) EWS/FLI","breast cancer, Infiltrating ductal carcinoma","ST & GR-ST are subclones of Tanoue (JCRB). GR-ST are ST cells transformed with human G-CSF receptor cDNA. Responds to G-CSF.","malignant embryonal carcinoma derived from lung","Human lymphoid cell line derived from Burkitt's lymphoma. Tumorigenic in nude mouse (meta from ileocecal lymph node)","Thymic carcinoma cell line.","","","","","HER2Amp; basal",""],
            
"patient_id":["PT-IMn7eqLZ","PT-pUBtKkYR","PT-El9YBuz5","PT-P6Iueia9", "PT-2gssvols", "PT-NczSWWPv", "PT-RY8p3mdI", "PT-olH0fsQe", "PT-3CksfnKs", "PT-OrIcgwcE", "PT-W3uUJEi1", "PT-B15oIzKN", "PT-fo7MYTEA", "PT-ley1jk9c", "PT-3f6vZIv9", "PT-MdR8meiJ", "PT-z5KcZzaT", "PT-3wNFFmJl", "PT-gpiijlSj", "PT-bdAvWDQz", "PT-uDGwdjJW", "PT-L63btI6L", "PT-uXJgRBuh", "PT-GkdLfVhc", "PT-UGLaOKH7", "PT-hw8KmxDu", "PT-G3kJesC0", "PT-zFOjyoHz", "PT-A8FG8EcM", "PT-98h5OTqP"]}

In [ ]:
a  = pd.DataFrame(toupdate)
a['name'] = samples.loc[notfound,"stripped_cell_line_name"].tolist()
a

In [ ]:
# updating..
for k, v in toupdate.items():
    samples.loc[notfound,k] =v

In [ ]:
# uploading to our bucket (now a new function)
a = changeToBucket(samples,'gs://cclebams/rna/', values=['internal_bam_filepath','internal_bai_filepath'], catchdup=False)

In [ ]:
mkdir temp

In [ ]:
# saving and updating the spreadsheet with these
print("YOU NOW NEED TO UPDATE THE GOOGLE SHEET!")
pd.concat([ccle_refsamples,samples],sort=False).to_csv('temp/updated_ccle_samples.csv')

In [ ]:
samples['arxspan_id'].tolist()

In [ ]:
samples = samples.rename(columns={'patient_id':'participant_id'})

In [ ]:
#pairs.participant_id = samples.participant_id

In [ ]:
#pairs = pairs.rename(columns={'patient_id':'participant_id'}).set_index('pair_id')

In [ ]:
#uploading new samples
refwm = refwm.disable_hound()
#refwm.upload_samples(samples)
#refwm.upload_entities('pairs', pairs)
#creating a sample set
refwm.update_sample_set(sample_set_id=samplesetname, sample_ids=samples.index)

## Check that we have all the cell lines we expect for this release
This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag'

compareToCuratedGS(url, sample = samples, samplesetname = samplesetname, colname = 'RNA New to internal')

# run the pipeline

We are using Dalmatian to send request to Terra, we are running a set of 6 functions To generate the expression/fusion dataset:

We use the GTEx pipeline ([https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md](https://github.com/broadinstitute/gtex-pipeline/blob/v9/TOPMed_RNAseq_pipeline.md)).

To generate the expression dataset, run the following tasks on all samples that you need, in this order:



*   samtofastq_v1-0_BETA_cfg 

    (broadinstitute_gtex/samtofastq_v1-0_BETA Snapshot ID: 5)

*   star_v1-0_BETA_cfg

(broadinstitute_gtex/star_v1-0_BETA Snapshot ID: 7)



*   rsem_v1-0_BETA_cfg 

    (broadinstitute_gtex/rsem_v1-0_BETA Snapshot ID: 4)

*   rsem_aggregate_results_v1-0_BETA_cfg (broadinstitute_gtex/rsem_aggregate_results_v1-0_BETA Snapshot ID: 3)

The outputs to be downloaded will be saved under the sample set that you ran. The outputs we use for the release are:



*   rsem_genes_expected_count
*   rsem_genes_tpm
*   rsem_transcripts_tpm

****Make sure that you delete the intermediate files. These files are quite large so cost a lot to store. To delete, you can either write a task that deletes them or use gsutil rm*****


##### Fusions {#fusions}

We use STAR-Fusion [https://github.com/STAR-Fusion/STAR-Fusion/wiki](https://github.com/STAR-Fusion/STAR-Fusion/wiki). The fusions are generated by running the following tasks



*   hg38_STAR_fusion (gkugener/STAR_fusion Snapshot ID: 14)
*   Aggregate_Fusion_Calls (gkugener/Aggregate_files_set Snapshot ID: 2)

The outputs to be downloaded will be saved under the sample set you ran. The outputs we use for the release are: 



*   fusions_star

This task uses the same samtofastq_v1-0_BETA_cfg task as in the expression pipeline, although in the current implementation, this task will be run twice. It might be worth combing the expression/fusion calling into a single workflow. This task also contains a flag that lets you specify if you want to delete the intermediates (fastqs). 

There are several other tasks in this workspace. In brief:



*   Tasks prefixed with **EXPENSIVE** or **CHEAP** are identical to their non-prefixed version, except that they specify different memory, disk space, etc. parameters. These versions can be used when samples fail the normal version of the task due to memory errors.
*   The following tasks are part of the GTEx pipeline but we do not use them (we use RSEM exclusively): markduplicates_v1-0_BETA_cfg (broadinstitute_gtex/markduplicates_v1-0_BETA Snapshot ID: 2), rnaseqc2_v1-0_BETA_cfg (broadinstitute_gtex/rnaseqc2_v1-0_BETA Snapshot ID: 2)
*   **ExonUsage_hg38_fixed** (gkugener/ExonUsage_fixed Snapshot ID: 1): this task calculates exon usage ratios. The non-fixed version contains a bug in the script that is not able to handle chromosome values prefixed with ‘chr’. The ‘fixed’ version resolves this issue.
*   **AggregateExonUsageRObj_hg38** (ccle_mg/AggregateExonUsageRObj Snapshot ID: 2): combines the exon usage ratios into a matrices that are saved in an R object.

## On Terra

In [ ]:
submission_id = refwm.create_submission("samtofastq_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id = refwm.create_submission("star_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, submission_id)

In [ ]:
submission_id1 = refwm.create_submission("rsem_v1-0_BETA_cfg", samplesetname,'sample_set',expression='this.samples')
terra.waitForSubmission(refworkspace, [submission_id1])

In [ ]:
# update a samples set with another sampleset
terra.updateAllSampleSet(refworkspace, samplesetname)

In [ ]:
submission_id1 = refwm.create_submission("rsem_aggregate_results", 'All_samples')

In [ ]:
submission_id3 = refwm.create_submission("hg38_STAR_fusion", samplesetname,'sample_set',expression='this.samples')

In [ ]:
submission_id_snp = refwm.create_submission("rnaseq-germline-snps-indels", samplesetname,'sample_set',expression='this.samples')

In [ ]:
terra.waitForSubmission(refworkspace, [submission_id1])#submission_id_snp])

In [ ]:
submission_id_snp = refwm.create_submission("merge_vcfs", 'All_samples')

## On Local

### Save the workflow configurations used

In [ ]:
terra.saveConfigs(refworkspace,'data/'+samplesetname+'/RNAconfig')

### Load QC files

In [ ]:
starlogs = getQC(workspace=refworkspace ,only=[], qcname="star_logs",match=".Log.final.out")
rnaqc = getQC(workspace=refworkspace ,only=[], qcname="rnaseqc2_metrics")

In [ ]:
new_ref = pd.read_csv('temp/newrefWES.csv',index_col="cds_sample_id")
for k,v in starlogs.items():
    if k =='nan':
        continue
    new_ref.loc[k,'bam_qc']=v+" "+rnaqc[k]
new_ref.to_csv('temp/newrefAll.csv')

### Remove some datafile to save money

In [ ]:
colstoremove = ['fastq1', 'fastq2', 'star_chimeric_bam_file','star_chimeric_bam_index', 'star_transcriptome_bam','recalibrated_bam','recalibrated_bam_index']
for val in colstoremove:
    refwm.disable_hound().delete_entity_attributes('sample', res[val], delete_files=True)

In [ ]:
res.iloc[1]['star_bam_file']

In [ ]:
## copy star bam file to our cclebams/rnasq_hg38/ bucket
renamed, _ = terra.changeGSlocation(workspacefrom=refworkspace, newgs="gs://cclebams/rnasq_hg38/", onlycol=["star_bam_file",'star_bam_index'], entity="sample", keeppath=False,dry_run=False)

In [ ]:
renamed

### Expression post processing

Here we get all data and remove the duplicates directly with the function `removeDuplicates`

we then run:

- readTranscripts
- readCounts
- readTPM
- renameFunction

- Allie's gene renaming / filtering and log transform

In [7]:
res = refwm.get_sample_sets().loc['All_samples']
rsem_genes_expected_count = res['rsem_genes_expected_count']
rsem_genes_tpm = res['rsem_genes_tpm']
rsem_transcripts_tpm = res['rsem_transcripts_tpm']
! gsutil cp $rsem_genes_expected_count "temp/expression.expectedcount" & gsutil cp $rsem_genes_tpm "temp/expression.genes.tpm" & gsutil cp $rsem_transcripts_tpm "temp/expression.transcript.tpm"

Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/751d1b66-5603-4056-a153-4570f01bc8d8/rsem_aggregate_results_workflow/b70ffcaf-b16e-4663-a64b-e87ddcba2605/call-rsem_aggregate_results/Aggregate_Fusion_Calls_2020-04-22T17-55-36.rsem_transcripts_tpm.txt.gz...
Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/751d1b66-5603-4056-a153-4570f01bc8d8/rsem_aggregate_results_workflow/b70ffcaf-b16e-4663-a64b-e87ddcba2605/call-rsem_aggregate_results/Aggregate_Fusion_Calls_2020-04-22T17-55-36.rsem_genes_tpm.txt.gz...
Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/751d1b66-5603-4056-a153-4570f01bc8d8/rsem_aggregate_results_workflow/b70ffcaf-b16e-4663-a64b-e87ddcba2605/call-rsem_aggregate_results/Aggregate_Fusion_Calls_2020-04-22T17-55-36.rsem_genes_expected_count.txt.gz...
\ [1 files][ 77.9 MiB/ 77.9 MiB]                                                
Operation completed over 1 objects/77.9 MiB.                                     
\ [1 files][ 74.7 MiB/ 74.7 MiB]  

In [8]:
for val in ["temp/expression.expectedcount"]:#,"temp/expression.transcript.tpm","temp/expression.genes.tpm"]:
    file = pd.read_csv(val, compression='gzip', header=0, sep='\t', quotechar='"', error_bad_lines=False)
    print(file.columns[:10])
    renaming,_ = removeOlderVersions(names=file.columns[2:], refsamples=refwm.get_samples(), arxspan_id="arxspan_id", version="version")
    file[file.columns[:2].tolist()+[i for i in file.columns if i in renaming.keys()]].rename(columns=renaming).to_csv(val, sep='\t', index=None)

Index(['gene_id', 'transcript_id(s)', 'CDS-010xbm', 'CDS-02TzJp', 'CDS-0693hw',
       'CDS-07Plat', 'CDS-0BNTL7', 'CDS-0CqPRM', 'CDS-0EOjym', 'CDS-0Jkjew'],
      dtype='object')
removed 65 duplicate samples
Index(['transcript_id', 'gene_id', 'CDS-010xbm', 'CDS-02TzJp', 'CDS-0693hw',
       'CDS-07Plat', 'CDS-0BNTL7', 'CDS-0CqPRM', 'CDS-0EOjym', 'CDS-0Jkjew'],
      dtype='object')
removed 65 duplicate samples
Index(['gene_id', 'transcript_id(s)', 'CDS-010xbm', 'CDS-02TzJp', 'CDS-0693hw',
       'CDS-07Plat', 'CDS-0BNTL7', 'CDS-0CqPRM', 'CDS-0EOjym', 'CDS-0Jkjew'],
      dtype='object')
removed 65 duplicate samples


#### saving samples used for 20Q2

In [ ]:
new_ref.loc[renaming.keys(),'20q2']=1
new_ref.to_csv('temp/newrefAll.csv')

In [28]:
%R install.packages('taigr')

R[write to console]: Installing package into ‘/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)



In [31]:
%%R
library('taigr')
source('src/load_libraries_and_annotations.R')
source('src/CCLE_postp_function.R')
library('cdsomics')

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Error in library(tidyverse) : there is no package called ‘tidyverse’
Calls: <Anonymous> ... withVisible -> source -> withVisible -> eval -> eval -> library




Error in library(tidyverse) : there is no package called ‘tidyverse’
Calls: <Anonymous> ... withVisible -> source -> withVisible -> eval -> eval -> library


In [10]:
%%R
# These files are downloaded from FireCloud/Terra
download_paths <- list(
  tpm_genes='temp/expression.genes.tpm',
  tpm_transcripts='temp/expression.transcript.tpm',
  counts_genes='temp/expression.expectedcount')

In [11]:
%%R
tpm_transcripts = readTranscripts(download_paths$tpm_transcripts)
counts_genes = readCounts(download_paths$counts_genes)
tpm_genes = readTPM(download_paths$tpm_genes)

R[write to console]: Error in readTranscripts(download_paths$tpm_transcripts) : 
  could not find function "readTranscripts"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in readTranscripts(download_paths$tpm_transcripts) : 
  could not find function "readTranscripts"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [12]:
%%R
head(tpm_transcripts)

R[write to console]: Error in head(tpm_transcripts) : object 'tpm_transcripts' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head




Error in head(tpm_transcripts) : object 'tpm_transcripts' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head


In [13]:
%%R
# Fix the colnames (for cases where there are mixed values (CCLE_name and DepMap_IDs))
colnames(counts_genes) %<>% renameFunction(.)
colnames(tpm_genes) %<>% renameFunction(.)
colnames(tpm_transcripts) %<>% renameFunction(.)

R[write to console]: Error in colnames(counts_genes) %<>% renameFunction(.) : 
  could not find function "%<>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in colnames(counts_genes) %<>% renameFunction(.) : 
  could not find function "%<>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


### data exploration and QC

- we QC on the amount of genes with 0 counts for each samples

In [14]:
%%R 
# Quick QC
# We are looking for samples with a worrying amount of zeros
zero_threshold <- 39000
number_zeros <- apply(tpm_genes[,2:ncol(tpm_genes)-1] ==0, 2, FUN = sum)
nzdf <- data.frame(CL=names(number_zeros), nz=number_zeros, stringsAsFactors = F)

R[write to console]: Error in apply(tpm_genes[, 2:ncol(tpm_genes) - 1] == 0, 2, FUN = sum) : 
  object 'tpm_genes' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> apply




Error in apply(tpm_genes[, 2:ncol(tpm_genes) - 1] == 0, 2, FUN = sum) : 
  object 'tpm_genes' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> apply


In [15]:
%%R
number_zeros <- number_zeros[order(-number_zeros)]
number_zeros <- number_zeros[number_zeros < zero_threshold]
pass <- number_zeros %>% names()

R[write to console]: Error in withVisible({ : object 'number_zeros' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in withVisible({ : object 'number_zeros' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [16]:
%%R
counts_genes

R[write to console]: Error in withVisible({ : object 'counts_genes' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in withVisible({ : object 'counts_genes' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [17]:
%%R
colnames(tpm_transcripts)[which(colnames(tpm_transcripts)=='transcript')] <- 'transcript_id(s)'
colnames(tpm_genes)[which(colnames(tpm_genes)=='transcript')] <- 'transcript_id(s)'
colnames(counts_genes)[which(colnames(counts_genes)=='transcript')] <- 'transcript_id(s)'

R[write to console]: Error in colnames(tpm_transcripts)[which(colnames(tpm_transcripts) ==  : 
  object 'tpm_transcripts' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in colnames(tpm_transcripts)[which(colnames(tpm_transcripts) ==  : 
  object 'tpm_transcripts' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [18]:
%%R
# These samples failed
failed <- setdiff(colnames(tpm_genes), pass) %>% .[!(. %in% c('gene_id', 'transcript_id(s)'))]

R[write to console]: Error in setdiff(colnames(tpm_genes), pass) %>% .[!(. %in% c("gene_id",  : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in setdiff(colnames(tpm_genes), pass) %>% .[!(. %in% c("gene_id",  : 
  could not find function "%>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [19]:
%%R
counts_genes

R[write to console]: Error in withVisible({ : object 'counts_genes' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in withVisible({ : object 'counts_genes' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [20]:
%%R
counts_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))

R[write to console]: Error in counts_genes %<>% dplyr::select(c("gene_id", "transcript_id(s)",  : 
  could not find function "%<>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in counts_genes %<>% dplyr::select(c("gene_id", "transcript_id(s)",  : 
  could not find function "%<>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [21]:
%%R
failed

R[write to console]: Error in withVisible({ : object 'failed' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in withVisible({ : object 'failed' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [22]:
%%R
head(tpm_transcripts)

R[write to console]: Error in head(tpm_transcripts) : object 'tpm_transcripts' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head




Error in head(tpm_transcripts) : object 'tpm_transcripts' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> head


In [23]:
%%R
pass <- setdiff(pass,c("gene_id","transcript_id(s)"))

R[write to console]: Error in as.vector(x) : object 'pass' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> setdiff -> as.vector




Error in as.vector(x) : object 'pass' not found
Calls: <Anonymous> -> <Anonymous> -> withVisible -> setdiff -> as.vector


In [24]:
%%R
tpm_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_transcripts %<>% dplyr::select(c("transcript_id", "gene_id", pass))

R[write to console]: Error in tpm_genes %<>% dplyr::select(c("gene_id", "transcript_id(s)",  : 
  could not find function "%<>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in tpm_genes %<>% dplyr::select(c("gene_id", "transcript_id(s)",  : 
  could not find function "%<>%"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [273]:
%%R library(ggplot)

UsageError: %%R is a cell magic, but the cell body is empty. Did you mean the line magic %R (single %)?


In [25]:
%%R
# Plot of the samples that fail
plot <- ggplot(nzdf, aes(nz)) +
  geom_histogram(bins = 100, color='black', fill='white') +
  geom_vline(xintercept = zero_threshold, linetype=2) +
  geom_label_repel(data = nzdf %>% filter(nz > zero_threshold), aes(x=nz, y=0, label=CL), size=5, fill=rgb(1,1,1,0.5))

ggsave(plot, filename ='data/zero_transcripts_distribution.png', width=20, height = 20)

R[write to console]: Error in ggplot(nzdf, aes(nz)) : could not find function "ggplot"
Calls: <Anonymous> -> <Anonymous> -> withVisible




Error in ggplot(nzdf, aes(nz)) : could not find function "ggplot"
Calls: <Anonymous> -> <Anonymous> -> withVisible


In [26]:
display(Image(filename='data/zero_transcripts_distribution.png'))

FileNotFoundError: [Errno 2] No such file or directory: 'temp/ggplot.png'

### Allie post processing 

In [ ]:
%%R
colnames(tpm_transcripts)[which(colnames(tpm_transcripts)=='transcript')] <- 'transcript_id(s)'
colnames(tpm_genes)[which(colnames(tpm_genes)=='transcript')] <- 'transcript_id(s)'
colnames(counts_genes)[which(colnames(counts_genes)=='transcript')] <- 'transcript_id(s)'

In [ ]:
%%R
print(colnames(tpm_genes)[1:10])
tpm_gene_ids <- gsub("\\..*", "", tpm_genes$gene_id)
if(nrow(tpm_genes) != length(unique(tpm_gene_ids))){
  print("Duplicated ensembl ids")
  print(nrow(tpm_genes) - length(unique(tpm_gene_ids)))
  tpm_genes <- tpm_genes[-which(duplicated(tpm_gene_ids)==T),]
}

In [ ]:
%%R
annotations<-download.raw.from.taiga(data.name='gene-annotations-e737', data.version=1, data.file='gencode.v29.annotation')
tpm_protein_coding <- prepare_depmap_TPM_for_taiga(tpm_genes, log_transform=T, just_protein_coding=T, gencode_annotations=annotations)
tpm_genes <- prepare_depmap_TPM_for_taiga(tpm_genes, log_transform=T, just_protein_coding=F, gencode_annotations=annotations)

In [ ]:
%%R
### transcripts
library('cdsomics')
print(colnames(tpm_transcripts)[1:10])
tpm_transcript_id <- gsub("\\..*", "", tpm_transcripts$`transcript_id`)
if(nrow(tpm_transcripts) != length(unique(tpm_transcript_id))) {
  print("Duplicated transcript ids")
  print(nrow(tpm_transcripts) - length(unique(tpm_transcript_id)))
  tpm_transcript <- tpm_transcripts[-which(duplicated(tpm_transcript_id)==T),]
}
tpm_transcripts <- prepare_depmap_transcripts_for_taiga(tpm_transcript, gencode_annotations = annotations) 

In [ ]:
%%R
### counts
print(colnames(counts_genes)[1:10])
counts_gene_ids <- gsub("\\..*", "", counts_genes$gene_id)
if(nrow(counts_genes) != length(unique(counts_gene_ids))) {
  print("Duplicate ensembl ids")
  print(length(which(duplicated(counts_gene_ids)==T)))
  counts_genes <- counts_genes[-which(duplicated(counts_gene_ids)==T),]
}
counts_genes <- prepare_depmap_TPM_for_taiga(counts_genes, gencode_annotations =annotations) 

In [ ]:
%%R
write.table(
  counts_genes, 
  file = paste0('temp/expression_', release,'_counts'), 
  sep = ',', quote = F)
write.table(
  tpm_genes, 
  file = paste0('temp/expression_', release,'_genes'), 
  sep = ',', quote = F)
write.table(
  tpm_protein_coding, 
  file = paste0('temp/expression_', release,'_protein_coding'), 
  sep = ',', quote = F)
write.table(
  tpm_transcripts, 
  file = paste0('temp/expression_', release,'_transcripts'),
  sep = ',', quote = F)

## Loading

In [192]:
counts_genes = pd.read_csv('temp/expression.'+ prevname + '.counts',index_col=0, sep='\t')
tpm_genes = pd.read_csv('temp/expression.'+ prevname + '.genes',index_col=0, sep='\t')
tpm_proteincoding = pd.read_csv('temp/expression.'+ prevname + '.protein_coding',index_col=0, sep='\t')
tpm_transcripts = pd.read_csv('temp/expression.'+ prevname + '.transcripts', index_col=0, sep='\t')

## ssGSEA

In [207]:
counts_genes.columns = [i.split(' (')[0] for i in counts_genes.columns]

In [208]:
len(set([val.split('.')[0] for val in counts_genes.columns if '.' in val]))

10910

In [255]:
#### merging splicing variants into the same gene
counts_genes_merged= mergeSplicingVariants(counts_genes.T,defined='.')

15%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



18%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



86%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [260]:
counts_genes_merged = counts_genes_merged[counts_genes_merged.sum(1)!=0]

In [262]:
counts_genes_merged = counts_genes_merged[counts_genes_merged.var(1)!=0]

### with R ssGSEA on the console

In [288]:
counts_genes_merged.to_csv("temp/for_ssGSEA.csv")

In [291]:
! Rscript src/ssGSEA.R 

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with


In [ ]:
enrichments = pd.read_csv('temp/res_ssGSEA.csv')

### with GSEApy

In [ ]:
genesets = ["GO_Biological_Process_2015", "GO_Cellular_Component_2015", "GO_Molecular_Function_2015", "Reactome_2013", "BioCarta_2013", "KEGG_2016", "CORUM", "TRANSFAC_and_JASPAR_PWMs", "Chromosome_Location","data/oncogenic.gmt",
"data/positional.gmt","data/hallmark.gmt", "data/cancer_modules.gmt"]

In [ ]:
counts_genes_merged['NAME'] = counts_genes_merged.index
counts_genes_merged = counts_genes_merged.drop(columns='genes')

In [ ]:
#for val in genesets:
res = gseapy.ssgsea(data=counts_genes_merged[counts_genes_merged.columns], gene_sets="KEGG_2016",
    outdir='temp/',
    sample_norm_method='rank', # choose 'custom' for your own rank list
    permutation_num=0, # skip permutation procedure, because you don't need it
    no_plot=True, # skip plotting, because you don't need these figures
    processes=8, 
    format='png',
    seed=9)

### with GSVA

In [101]:
! Rscript -e 'install.packages("BiocManager");\
              BiocManager::install(c("GSEABase","GSVA"),dep=TRUE)'
! pip install GSVA

Installing package into ‘/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)
trying URL 'https://cloud.r-project.org/src/contrib/BiocManager_1.30.10.tar.gz'
Content type 'application/x-gzip' length 40205 bytes (39 KB)
downloaded 39 KB

* installing *source* package ‘BiocManager’ ...
** package ‘BiocManager’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (BiocManager)

The downloaded source packages are in
	‘/tmp/Rtmp58v802/downloaded_packages’
Bioconductor version 3.9 (BiocManager 1.30.10), R 3.6.1 (2019-07-05)
Installing package(s) 'GSEABase', 'GSVA'
also installing the

Content type 'application/x-gzip' length 13351270 bytes (12.7 MB)
downloaded 12.7 MB

trying URL 'https://bioconductor.org/packages/3.9/bioc/src/contrib/GSEABase_1.46.0.tar.gz'
Content type 'application/x-gzip' length 240269 bytes (234 KB)
downloaded 234 KB

trying URL 'https://bioconductor.org/packages/3.9/bioc/src/contrib/GSVA_1.32.0.tar.gz'
Content type 'application/x-gzip' length 1554886 bytes (1.5 MB)
downloaded 1.5 MB

* installing *source* package ‘dichromat’ ...
** package ‘dichromat’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** data
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (dichromat)
* installing *source* package ‘biomaRt’ ...
** using staged install

                 from cliques.cpp:1:
/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/smart_ptr/detail/shared_count.hpp:356:33: warning: ‘template<class> class std::auto_ptr’ is deprecated [-Wdeprecated-declarations]
     explicit shared_count( std::auto_ptr<Y> & r ): pi_( new sp_counted_impl_p<Y>( r.get() ) )
                                 ^~~~~~~~
In file included from /usr/include/c++/6/memory:81:0,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/config/no_tr1/memory.hpp:21,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/smart_ptr/shared_ptr.hpp:23,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/shared_ptr.hpp:17,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/vector_property_map.hpp:14,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/property_m

In file included from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/shared_ptr.hpp:17:0,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/vector_property_map.hpp:14,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/property_map.hpp:602,
                 from RBGL.hpp:17,
                 from clusteringCoef.cpp:1:
/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/smart_ptr/shared_ptr.hpp:471:31: warning: ‘template<class> class std::auto_ptr’ is deprecated [-Wdeprecated-declarations]
     explicit shared_ptr( std::auto_ptr<Y> & r ): px(r.get()), pn()
                               ^~~~~~~~
In file included from /usr/include/c++/6/memory:81:0,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/config/no_tr1/memory.hpp:21,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/smart_ptr/s

                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/property_map.hpp:602,
                 from RBGL.hpp:17,
                 from dominatorTree.cpp:1:
/usr/include/c++/6/bits/unique_ptr.h:49:28: note: declared here
   template<typename> class auto_ptr;
                            ^~~~~~~~
In file included from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/shared_ptr.hpp:17:0,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/vector_property_map.hpp:14,
                 from /home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/property_map/property_map.hpp:602,
                 from RBGL.hpp:17,
                 from dominatorTree.cpp:1:
/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/BH/include/boost/smart_ptr/shared_ptr.hpp:567:34: warning: ‘template<class> class std::auto_ptr’ is deprecated [-Wdeprecated-declarations]
     shared_ptr & operator=( std::auto

...

gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG    -I/tmp/Rtmpx0jjFQ/R.INSTALL6b0a6a73e0a2/Rgraphviz/src/libunix/include/graphviz -DGRAPHVIZ_MAJOR=2 -DGRAPHVIZ_MINOR=28 -DGRAPHVIZ_STATIC -DHAVE_STDBOOL_H=1 -DHAVE_BOOL=1 -fpic  -g -O2 -fdebug-prefix-map=/home/jranke/git/r-backports/stretch/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c RgraphvizInit.c -o RgraphvizInit.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG    -I/tmp/Rtmpx0jjFQ/R.INSTALL6b0a6a73e0a2/Rgraphviz/src/libunix/include/graphviz -DGRAPHVIZ_MAJOR=2 -DGRAPHVIZ_MINOR=28 -DGRAPHVIZ_STATIC -DHAVE_STDBOOL_H=1 -DHAVE_BOOL=1 -fpic  -g -O2 -fdebug-prefix-map=/home/jranke/git/r-backports/stretch/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c agopen.c -o agopen.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG    -I/tmp/Rtmpx0jjFQ/R.INSTALL6b0a6a73e0a2/Rgraphviz/src/libunix/include/graphviz -DGRAPH

g++ -std=gnu++11 -I"/usr/share/R/include" -DNDEBUG  -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/Rcpp/include"   -fpic  -g -O2 -fdebug-prefix-map=/home/jranke/git/r-backports/stretch/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c R_fit_levenberg.cpp -o R_fit_levenberg.o
g++ -std=gnu++11 -I"/usr/share/R/include" -DNDEBUG  -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/Rcpp/include"   -fpic  -g -O2 -fdebug-prefix-map=/home/jranke/git/r-backports/stretch/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c R_fit_one_group.cpp -o R_fit_one_group.o
g++ -std=gnu++11 -I"/usr/share/R/include" -DNDEBUG  -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/Rcpp/include"   -fpic  -g -O2 -fdebug-prefix-map=/home/jranke/git/r-backports/stretch/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c R_get_one_way_f

* installing *source* package ‘Category’ ...
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (Category)
* installing *source* package ‘bookdown’ ...
** package ‘bookdown’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (bookdown)
* installing *source* package ‘GSVAdata

gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG `echo 'Rhtslib::pkgconfig("PKG_CPPFLAGS")'| "/usr/lib/R/bin/R" --vanilla --slave` -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/S4Vectors/include" -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/IRanges/include" -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/XVector/include" -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/Biostrings/include" -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/Rhtslib/include"   -fpic  -g -O2 -fdebug-prefix-map=/home/jranke/git/r-backports/stretch/r-base-3.6.1=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c vcffile.c -o vcffile.o
gcc -std=gnu99 -I"/usr/share/R/include" -DNDEBUG `echo 'Rhtslib::pkgconfig("PKG_CPPFLAGS")'| "/usr/lib/R/bin/R" --vanilla --slave` -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/S4Vectors/include" -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3.6/IRanges/include" -I"/home/jeremie/R/x86_64-pc-linux-gnu-library/3

** testing if installed package keeps a record of temporary installation path
* DONE (ggbio)
* installing *source* package ‘ReportingTools’ ...
** using staged installation
** R
** data
** inst
** byte-compile and prepare package for lazy loading
in method for 'objectToHTML' with signature 'object="ggbio"': no definition for class "ggbio"
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (ReportingTools)

The downloaded source packages are in
	‘/tmp/Rtmp58v802/downloaded_packages’
Installation path not writeable, unable to update packages: boot, class,
  KernSmooth, lattice, MASS, Matrix, mgcv, nlme, nnet, spatial, survival
Old packages: 'attempt', 'backports', 'BH', 'bit', 'bit64', 'blob', 'broom',
  'callr', 'checkmate', 'cli', '

In [104]:
from GSVA import gsva, gmt_to_dataframe

In [115]:
genesets = ! ls data/genesets/*

In [116]:
genesets

['data/genesets/BioCarta_2013.gmt',
 'data/genesets/cancer_modules.gmt',
 'data/genesets/chr_arms.gmt',
 'data/genesets/Chromosome_Location.gmt',
 'data/genesets/CORUM.gmt',
 'data/genesets/GDSC_target.gmt',
 'data/genesets/GO_Biological_Process_2015.gmt',
 'data/genesets/GO_Cellular_Component_2015.gmt',
 'data/genesets/GO_Molecular_Function_2015.gmt',
 'data/genesets/hallmark.gmt',
 'data/genesets/KEGG_2016.gmt',
 'data/genesets/oncogenic.gmt',
 'data/genesets/positional.gmt',
 'data/genesets/Reactome_2013.gmt',
 'data/genesets/TRANSFAC_and_JASPAR_PWMs.gmt']

In [297]:
a = pd.read_csv('data/genesets/ALL.gmt',sep='\t', names = range(20))

In [302]:
h.dups(a[0])

['chr6q',
 'chr1q41',
 'chr8p23',
 'chr3q27',
 'chr13q33',
 'chr17q21',
 'chr9p',
 'chr10q',
 'chr5q23',
 'chr5p14',
 'chr18p11',
 'chr16p13',
 'chr4q13',
 'chr17p',
 'chr3p22',
 'chr15q26',
 'chr22q13',
 'chr4q21',
 'chr11q25',
 'chr13q32',
 'chr2q35',
 'chr1p32',
 'chr5p12',
 'chr16q11',
 'chr5q34',
 'chr21q',
 'chr2p12',
 'chr2p13',
 'chr20q',
 'chr4q23',
 'chr3p11',
 'chr7p',
 'chr2q37',
 'chr5q15',
 'chr2p16',
 'chr11p',
 'chr3p12',
 'chr21p',
 'chr10q26',
 'chr10q21',
 'chr5q22',
 'chr3q22',
 'chr6q13',
 'chr11q22',
 'chr2q',
 'chr5p15',
 'chr7p11',
 'chr3q12',
 'chr13q34',
 'chr2q11',
 'chr6q12',
 'chr10q11',
 'chr16p12',
 'chr13q11',
 'chr20p12',
 'chr2q21',
 'chr19q',
 'chr3q21',
 'chr4q12',
 'chr10p14',
 'chr1q25',
 'chr18q22',
 'chr11p13',
 'chr1p31',
 'chr11q23',
 'chr2q22',
 'chr9p12',
 'chr12p12',
 'chr12q15',
 'chr4q',
 'chr10q22',
 'chr4p11',
 'chr1p33',
 'chr8q11',
 'chr19p',
 'chr6p',
 'chr3p24',
 'chr10p13',
 'chr1q44',
 'chr2p21',
 'chr1q22',
 'chr9p23',
 'chr7p13',

In [292]:
gmt = gmt_to_dataframe("data/genesets/ALL.gmt")

In [124]:
counts_genes_merged.index.name = "gene_name"

In [132]:
gsva(counts_genes_merged, gmt, method='ssgsea', verbose=True)

Caching to /tmp/weirathe.0_jn6djc
WARNING removing 135 genes from gene sets that don't exist in the data
ABI-2,ADPRT,APM1,ARAF1,ARHA,ASS,ATP5A1,BDK,BF,BG1,BIG1,BIRC4,BUCS1,C1QG,CD3Z,CDC2,CDC2L1,CDC2L2,CENTD1,CENTD2,CHC1,CNK,COP,CRN,CSEN,CYP3A,DDEF1,DDEF2,DF,DSPG3,DTR,EDG1,EDG2,EDG4,EDG7,EIF1A,EIF3S10,ELA1,ELA2,ELA2A,ELA2B,ELA3B,EPLIN,ERAF,FAT,FOXO1A,FOXO3A,FRA8B,FRAP1,G22P1,GAPD,GLI,GPRK2L,GRF2,HBXIP,HRI,HSPCA,HTATIP,ICOSL,IKBKAP,IL14,IL17,IL8,ISGF3G,KNG,KRAS2,MADH1,MADH2,MADH3,MADH4,MADH5,MADH6,MADH7,MADHIP,MAP3K7IP1,MAP3K7IP2,MEL,MLLT7,MRE11A,MTCO1,NBS1,NGFB,NOS2A,PARK2,PCAF,PCNT1,PCNT2,PDCD8,PDZGEF1,PFC,PFKX,PGLYRP,PLK,PNUTL1,PPARBP,PPARGC1,PRAM-1,PRKCB1,PRKCL1,PRKR,PSCD1,PSCD2,PSCD3,PSCD4,PTCH,RAB2,RAB7,RBPSUH,REA,SHARP,SITPEC,SKIP,SKP1A,TCF1,TIP-1,TNFRSF5,TNFRSF6,TNFSF5,TNFSF6,TNS,TRA@,TRB@,TUBA1,TUBA2,TUBA3,TUBA4,TUBA6,UBE1,UBL1,VEGF,VIL2,WBSCR14,ZNF145,ZNF42,ZNFN1A3
Rscript /home/jeremie/miniconda3/lib/python3.7/site-packages/GSVA/gsva.r ssgsea Gaussian False 1 None 0 SOCK True 

UnsupportedOperation: fileno

## validation

In [ ]:
unmatched = getNonZeroCorrelations(counts_genes ,prevcounts)

In [ ]:
unmatched

In [ ]:
# Is it because of  duplicate version?
rnasamples = ccle_refsamples[ccle_refsamples.datatype=='rna']
for i,val in unmatched:
    print(len(rnasamples[rnasamples.arxspan_id==i]))

In [ ]:
# <20million (~10million)
removedfromPiPelineNotEnoughReads = ['CDS-ABH0uZ','CDS-fk564T','CDS-kU30H5']

In [ ]:
notinnew = set(prevcounts.index) - set(counts_genes.index)
notinnew

In [ ]:
removedFromListNotEnoughExpressedGenes = ["ACH-001577", "ACH-001686", "ACH-002463", "ACH-002055", "ACH-001388", "ACH-000052"]

### replacing different values

In [ ]:
counts_genes = counts_genes.T
tpm_genes = tpm_genes.T
tpm_proteincoding = tpm_proteincoding.T
tpm_transcripts = tpm_transcripts.T

In [ ]:
for k, _ in unmatched+[('ACH-000561',''),('ACH-000052','')]:
    counts_genes[k] = prevcounts.loc[k].values
    tpm_genes[k] = prevgenes.loc[k].values
    tpm_proteincoding[k] = prevproteincoding.loc[k].values
    tpm_transcripts[k] = prevtranscripts.loc[k].values

In [ ]:
ls temp/

In [ ]:
len(tpm_proteincoding.index)

In [ ]:
set(prevproteincoding.index) - set(tpm_proteincoding.index)

In [ ]:
set(prevproteincoding.index) - set(tpm_proteincoding.index)

In [ ]:
prevversion

In [ ]:
#prevcounts = tc.get(name='depmap-rnaseq-expression-data-363a', version=24, file="internal_"+prevname+'_counts')
prevgenes = tc.get(name='depmap-rnaseq-expression-data-363a', version=24, file="internal_"+prevname+'_tpm')
prevtranscripts = tc.get(name='depmap-rnaseq-expression-data-363a', version=24, file="internal_"+prevname+'_transcripts_tpm')
prevproteincoding = tc.get(name='depmap-rnaseq-expression-data-363a', version=24, file="internal_"+prevname+'_proteincoding_tpm')

### do we have the same cell lines as before

In [ ]:
len(prevcounts), len(counts_genes)

In [ ]:
set(prevproteincoding.index) - set(counts_genes.index)

In [ ]:
counts_genes
tpm_genes
tpm_proteincoding
tpm_transcripts

In [ ]:
tpm_proteincoding = pd.concat([tpm_proteincoding,  prevproteincoding.loc[['ACH-000052', 'ACH-000561']]])
tpm_transcripts = pd.concat([tpm_transcripts, prevtranscripts.loc[['ACH-000052', 'ACH-000561']]])
tpm_genes = pd.concat([tpm_genes, prevgenes.loc[['ACH-000052', 'ACH-000561']]])
counts_genes = pd.concat([counts_genes, prevcounts.loc[['ACH-000052', 'ACH-000561']]])

### do we have new duplicates that don't correlate well and why?

In [ ]:
def getNonZeroCorrelations(df1,df2):
    res=[]
    for k,val in df1.iterrows():
        if k in df2.index:
            corr = np.corrcoef(df1.loc[k],df2.loc[k])
            if corr[0,1]<0.99999999999999:
                res.append((k,corr[0,1]))
        else:
            print(k+" not in second df")
    print("found "+str(len(res))+" samples that did not match")
    return res

In [ ]:
counts_genes.values.shape

In [ ]:
unmatched = getNonZeroCorrelations(counts_genes ,prevcounts)

In [ ]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(counts_genes))
counts_genes = counts_genes[~counts_genes.index.isin(blacklist)]
print(len(counts_genes))
#counts_genes.to_csv('temp/internal_'+release+'_counts', sep='\t')
print(len(tpm_genes))
tpm_genes = tpm_genes[~tpm_genes.index.isin(blacklist)]
print(len(tpm_genes))
#tpm_genes.to_csv('temp/internal_'+release+'_tpm', sep='\t')
print(len(tpm_proteincoding))
tpm_proteincoding = tpm_proteincoding[~tpm_proteincoding.index.isin(blacklist)]
print(len(tpm_proteincoding))
#tpm_proteincoding.to_csv('temp/internal_'+release+'_proteincoding_tpm', sep='\t')
print(len(tpm_transcripts))
tpm_transcripts = tpm_transcripts[~tpm_transcripts.index.isin(blacklist)]
print(len(tpm_transcripts))
#tpm_transcripts.to_csv('temp/internal_'+release+'_transcripts_tpm', sep='\t')len(unmatched)

In [ ]:
ACH-001515 new reprocessing 0.9946915477726004 MERGE
ACH-001520 new reprocessing 0.9957699212101433 MERGE
ACH-001740 new version (but first one seems much better in terms of reads, second one seems much better in RNAseqQC) 0.8015848372375373 TAKE SECOND
ACH-001608 new version 0.9945092691624465 MERGE
ACH-001334 new version same as 1740 but first one has much more reads TAKE FIRST 0.9767280911431145 MERGE
ACH-001745 new version same as 1740 0.9829779076971114 MERGE
ACH-000206 new version same as 1740 but in the end the total amount of mapped read is better in second version 0.9872061994453488 MERGE
ACH-001841 new version 0.9891607464895766 MERGE
ACH-001541 new version 0.9903054150416472 MERGE
ACH-000597 new version 0.9807940519711182 MERGE
ACH-000143 new version 0.9775750844260739 MERGE
ACH-001765 new version 0.9665153948393865 MERGE
ACH-001422 new version 0.993737865220167 MERGE
ACH-000095 new version same as 1740 0.9307949055051599 TAKE FIRST
ACH-000468 0.9574272807923139 TAKE FIRST
ACH-001192 0.9786159108766187 MERGE
ACH-000556 0.9658561991609396 MERGE
ACH-000700 0.9724575358855265 MERGE
ACH-001495 0.9933487615618818 MERGE
ACH-000975 0.9670679001304227 MERGE
ACH-000517 0.9674345128976869 MERGE
ACH-000337 0.975427126351326 MERGE
ACH-000029 first seems a bit better quality and more reads 0.9740794074881175 MERGE
ACH-000455 0.9694405381565491 MERGE
ACH-001321 0.720323357250316 BOTH are the same cell line but GIVEN how bad these lines are, I would advice to use the newest version nonetheless as it is almost the same as tthe first one in terms of read counts and bettter in QC
ACH-000328 0.9716845943633539 MERGE

merge everything above .96

In [ ]:
unmatched

In [ ]:
# Is it because of  duplicate version?
rnasamples = ccle_refsamples[ccle_refsamples.datatype=='rna']
for i,val in unmatched:
    print(len(rnasamples[rnasamples.arxspan_id==i]))

In [ ]:
# <20million (~10million)
removedfromPiPelineNotEnoughReads = ['CDS-ABH0uZ','CDS-fk564T','CDS-kU30H5']

In [ ]:
notinnew = set(prevcounts.index) - set(counts_genes.index)
notinnew

In [ ]:
removedFromListNotEnoughExpressedGenes = ["ACH-001577", "ACH-001686", "ACH-002463", "ACH-002055", "ACH-001388", "ACH-000052"]

### replacing different values

In [ ]:
counts_genes = counts_genes.T
tpm_genes = tpm_genes.T
tpm_proteincoding = tpm_proteincoding.T
tpm_transcripts = tpm_transcripts.T

In [ ]:
for k, _ in unmatched+[('ACH-000561',''),('ACH-000052','')]:
    counts_genes[k] = prevcounts.loc[k].values
    tpm_genes[k] = prevgenes.loc[k].values
    tpm_proteincoding[k] = prevproteincoding.loc[k].values
    tpm_transcripts[k] = prevtranscripts.loc[k].values

In [ ]:
counts_genes = counts_genes.T
tpm_proteincoding = tpm_proteincoding.T
tpm_genes = tpm_genes.T
tpm_transcripts = tpm_transcripts.T

### Some predictions

In [ ]:
metadata = tc.get(name='depmap-a0ab', file='sample_info')

In [ ]:
metadata

In [ ]:
# finding train and test set
trainame = [val for val in new1&prev if val[:3] == 'ACH']
testname = [val for val in new1-prev if val[:3] == 'ACH']

#looking at the 2000 most variable genes in the two sets
genetolookfor = 2000
gene_var = counts_genes[trainame].var(1).values
print(len(gene_var))
sorting = np.argsort(gene_var)[-genetolookfor:]

In [ ]:
unregistered = set(testname) - set(metadata["DepMap_ID"].values.tolist())
unregistered

In [ ]:
len(counts_genes['ACH-001767']) - np.count_nonzero(counts_genes['ACH-001767'].values)

In [ ]:
# creating and reodering train and test sets
traindata = counts_genes[set(trainame)-unregistered].values[sorting].T
trainlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(trainame)-unregistered].columns.tolist() if val not in unregistered]

testdata = counts_genes[set(testname)-unregistered].values[sorting].T
testlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(testname)-unregistered].columns.tolist() if val not in unregistered]

In [ ]:
traindata.shape

In [ ]:
# learn KNN classifier to the metadata diseases
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(traindata, trainlabels) 
predicted = neigh.predict(testdata)

In [ ]:
predicted[1]

In [ ]:
labels = trainlabels + testlabels
colors=[0]*len(trainlabels)
colors.extend([1,2,2,2,2,1,2,2,2,1,2])
data = np.vstack([traindata,testdata])

In [ ]:
# plot them with TSNE, highlight the points that failed and show colors for diseases
dimred = TSNE(2,10).fit_transform(data)

In [ ]:
scatter(dimred, labels=labels,colors=colors, radi=1.9)

## Save files for taiga

In [ ]:
! ls temp/expression.*

In [ ]:
# FOR 20Q3 KNOW THAT WE WILL RETRIEVE DATA FROM 20Q2 WITH THE VERY DFFERENT CELL LINES!!

In [ ]:
a = pd.read_csv('temp/expression_20Q2_genes',sep='\t')

In [ ]:
a.loc['ACH-001632']

In [ ]:
tc.update_dataset(dataset_permaname="depmap-expression-87f8",
                 upload_file_path_dict={'temp/expression_'+release+'_transcripts': 'NumericMatrixCSV',
                                       'temp/expression_'+release+'_genes': 'NumericMatrixCSV',
                                       'temp/expression_'+release+'_counts': 'NumericMatrixCSV',
                                       'temp/expression_'+release+'_protein_coding': 'NumericMatrixCSV'},
                  dataset_description=
"""
# RNAseq

Combined segment and gene-level CN calls from Broad WES, Sanger WES, and Broad SNP. Relative CN, not log2 transformed.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1-8: guillaume releases
version 9: 19Q3 release
version 10:  adding missing samples in Terra merge files
version 11: 19Q4 new release.

Adding 93 new cell lines. 
Some cells lines have been removed because they:

 - had too many 0 values = ["ACH-001388" "ACH-001577" "ACH-001767" "ACH-002463"] 

Some cells lines have been flagged as:

 - having too many..
 
version 12:
    uploading as matrices 
    
version 13:
    20Q1 new release. 
    
version 14:
    20Q2 new release, adding 50 new samples
 
transcriptions (Transcripts rpkm):

genes (gene rpkm):
__Rows__:
__Columns__:
Counts (gene counts):
__Rows__:
__Columns__:
Gene level CN data:
__Rows__:
__Columns__:
 DepMap cell line IDs
 gene names in the format HGNC\_symbol (Entrez\_ID)
DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
""")

In [79]:
rna_dmc_embargo

['ACH-001024',
 'ACH-001036',
 'ACH-001293',
 'ACH-001349',
 'ACH-001385',
 'ACH-001429',
 'ACH-001437',
 'ACH-001438',
 'ACH-001449',
 'ACH-001493',
 'ACH-001502',
 'ACH-001537',
 'ACH-001547',
 'ACH-001577',
 'ACH-001578',
 'ACH-001603',
 'ACH-001620',
 'ACH-001626',
 'ACH-001662',
 'ACH-001669',
 'ACH-001672',
 'ACH-001676',
 'ACH-001678',
 'ACH-001686',
 'ACH-001693',
 'ACH-001696',
 'ACH-001703',
 'ACH-001716',
 'ACH-001767',
 'ACH-001820',
 'ACH-001854',
 'ACH-001855',
 'ACH-001970',
 'ACH-001971',
 'ACH-001973',
 'ACH-002010',
 'ACH-002021',
 'ACH-002048',
 'ACH-002055',
 'ACH-002065',
 'ACH-002462',
 'ACH-002463',
 'ACH-002464',
 'ACH-002465',
 'ACH-002466',
 'ACH-002467',
 'ACH-002475',
 'ACH-002508',
 'ACH-002510',
 'ACH-002511',
 'ACH-002512',
 'ACH-002709']

In [78]:
gsheets = sheets.get(privacy_release_url).sheets[6].to_frame()
rna_ibm_embargo = [i for i in gsheets['RNAseq_IBM_embargo'].values.tolist() if str(i) != 'nan']
rna_dmc_embargo = [i for i in gsheets['RNAseq_DMC_embargo'].values.tolist() if str(i) != 'nan']
blacklist = [i for i in gsheets['blacklist'].values.tolist() if str(i) != 'nan']

Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jeremie/miniconda3/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handlin

In [ ]:
blacklist, rna_dmc_embargo, rna_ibm_embargo

### internal

In [ ]:
tpm_genes.shape

In [ ]:
len(tpm_genes)

In [ ]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(counts_genes))
counts_genes = counts_genes[~counts_genes.index.isin(blacklist)]
print(len(counts_genes))
counts_genes.to_csv('temp/internal_'+release+'_counts')
print(len(tpm_genes))
tpm_genes = tpm_genes[~tpm_genes.index.isin(blacklist)]
print(len(tpm_genes))
tpm_genes.to_csv('temp/internal_'+release+'_tpm')
print(len(tpm_proteincoding))
tpm_proteincoding = tpm_proteincoding[~tpm_proteincoding.index.isin(blacklist)]
print(len(tpm_proteincoding))
tpm_proteincoding.to_csv('temp/internal_'+release+'_proteincoding_tpm')
print(len(tpm_transcripts))
tpm_transcripts = tpm_transcripts[~tpm_transcripts.index.isin(blacklist)]
print(len(tpm_transcripts))
tpm_transcripts.to_csv('temp/internal_'+release+'_transcripts_tpm')

In [ ]:
prev_counts_genes = tc.get(name='depmap-rnaseq-expression-data-363a', version=26, file='internal_'+prevname+'_counts')
print('shoud be None')
print(set(prev_counts_genes.index) - set(counts_genes.index))
print("new lines")
newlines = set(counts_genes.index) - set(prev_counts_genes.index) 
newlines

In [ ]:
tc.update_dataset(dataset_permaname="depmap-rnaseq-expression-data-363a",
                 upload_file_path_dict={'temp/internal_'+release+'_counts': 'NumericMatrixCSV',
                                       'temp/internal_'+release+'_tpm': 'NumericMatrixCSV',
                                       'temp/internal_'+release+'_proteincoding_tpm': 'NumericMatrixCSV',
                                       'temp/internal_'+release+'_transcripts_tpm': 'NumericMatrixCSV'},
                  dataset_description=
"""
# INTERNAL RNA

* Version 1-3 Internal 18Q1*

All CCLE cell lines with RNAseq data.

Original data source:

`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_depMap_18Q1_RNAseq_reads_20180201.gct`
`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_depMap_18Q1_RNAseq_RPKM_20180201.gct`

Version 2 of RPKM data are log2-transformed with a noisy floor at -3 (-3 + N(0, 0.1))

* Version 4-6 Internal 18Q2*

Original data source:

`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_depMap_18q2_RNAseq_reads_20180420.gct` `/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_depMap_18q2_RNAseq_RPKM_20180420.gct`

RPKM data are log2-transformed with a noisy floor at -3 (-3 + N(0, 0.1)). Reads file unaltered aside from formatting row / column names.

* Version 7 Internal 18Q2*

Includes a matrix with genes filtered by HGNC protein-coding gene locus group.

* Version 8-10 Internal 18Q3*

use version 10

Original data source:

`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_depMap_18q3_RNAseq_reads_20180716.gct` `/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_depMap_18q3_RNAseq_RPKM_20180716.gct`

RPKM data are log2-transformed with a noisy floor at -3 (-3 + N(0, 0.1)). Reads file unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are Broad (Arxspan) cell line IDs.

Columns: In the complete RPKM and read datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding RPKM and read datasets column names are HGNC_symbol (Entrez_ID)

version 9 updates names, and slightly different RPKM values due to randomly added noisy floor (using a seed of 4)

version 10 removes duplicate gene names from the protein coding datasets

* Version 11-12 Internal 18Q4*

18Q4 transcript level data is found in version 14. (In versions 1-13 transcript data contains only gene level not transcript level data)

changing to TPM expression

Original data source:

`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_DepMap_18Q4_rsem_genes_tpm_20181029.txt` `/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_DepMap_18Q4_rsem_transcripts_tpm_20181029.gct` `/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_DepMap_18Q4_RNAseq_reads_20181029.gct` `/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_DepMap_18Q4_RNAseq_RPKM_20181029.gct`

TPM data is the primary expression data now. It is log2-transformed with a pseudo count of 1 added. The TPM data contains 4 cell lines not included in the RPKM data.

RPKM data are log2-transformed with a pseudo count of 1 added. RPKM values are no longer thresholded.

Reads/transcript files unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are DepMap (Arxspan) cell line IDs

Columns: In the complete TPM, TPM transcripts, RPKM and read datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding TPM dataset column names are HGNC_symbol (Entrez_ID)

* Version 13-15 Internal 19Q1*

Version 15 contains the correct data sets for 19Q1 - 2 cell lines are removed

Version 14 contains the correct transcript level data for 18Q4

* Version 16 Internal 19Q2*

* Version 17 Internal 19Q3*

* Version 18 Internal 19Q4*

Adding 93 new cell lines - Blacklisted
Some cells lines have been removed because they:

 - had too many 0 values = ["ACH-001388" "ACH-001577" "ACH-001767" "ACH-002463"] 

Some cells lines have been flagged as:

 - None
* Version 19 Internal 19Q4
removing blacklisted

* Version 20 Internal 19Q4
removing blacklisted in transcripts

* Version 21 Internal 19Q4
uploading as matrices 

* Version 22 Internal 20Q1
adding 6 new cell lines

* Version 23 Internal 20Q2
adding  new cell lines

* Version 24 Internal 20Q2
adding  back ACH-000052

* Version 25 Internal 20Q3
nothing different from  20Q2. no new cell lines added

* Version 26 Internal 20Q3
some lines were wrongly added to the blacklist

data is aligned to hg38

read count data is created using RSEM, which, when a read maps to multiple places, splits the counts between genes, with the weight based on the likelihood that it came from one gene or the other, so counts data may not be integers

TPM data is log2-transformed with a pseudo count of 1 added. log2(X+1)

Reads/transcript files unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are DepMap cell line IDs Mapping between Broad IDs and CCLE IDs can be done using a R or python package

To install R implementation: options(repos = c("https://iwww.broadinstitute.org/~datasci/R-packages", "https://cran.cnr.berkeley.edu")) install.packages('celllinemapr')

To install python implementation: pip install https://intranet.broadinstitute.org/~datasci/python-packages/cell_line_mapper-0.1.9.tar.gz)

Columns: In the complete TPM and read counts datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding TPM dataset column names are HGNC_symbol (Entrez_ID). In the TPM transcript dataset column names are HGNC_symbol (Transcript_ID) - the HGNC symbols are not unique, use the transcript IDs for unique identifiers.


NEW LINES:
"""+newlines)

In [ ]:
AddToVirtual('depmap-a0ab', "depmap-rnaseq-expression-data-363a", files=[('CCLE_RNAseq_reads', 'internal_'+release+'_counts'),('CCLE_expression_full', 'internal_'+release+'_tpm'),('CCLE_expression', 'internal_'+release+'_proteincoding_tpm'),('CCLE_RNAseq_transcripts', 'internal_'+release+'_transcripts_tpm')])

AddToVirtual(virtual_internal, "depmap-rnaseq-expression-data-363a", files=[('CCLE_RNAseq_reads', 'internal_'+release+'_counts'),('CCLE_expression_full', 'internal_'+release+'_tpm'),('CCLE_expression', 'internal_'+release+'_proteincoding_tpm'),('CCLE_RNAseq_transcripts', 'internal_'+release+'_transcripts_tpm')])

### IBM

like internal

### DMC

In [22]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(counts_genes))
counts_genes = counts_genes[~counts_genes.index.isin(rna_ibm_embargo)]
print(len(counts_genes))
counts_genes.to_csv('temp/dmc_'+release+'_counts')
print(len(tpm_genes))
tpm_genes = tpm_genes[~tpm_genes.index.isin(rna_ibm_embargo)]
print(len(tpm_genes))
tpm_genes.to_csv('temp/dmc_'+release+'_tpm')
print(len(tpm_proteincoding))
tpm_proteincoding = tpm_proteincoding[~tpm_proteincoding.index.isin(rna_ibm_embargo)]
print(len(tpm_proteincoding))
tpm_proteincoding.to_csv('temp/dmc_'+release+'_proteincoding_tpm')
print(len(tpm_transcripts))
tpm_transcripts = tpm_transcripts[~tpm_transcripts.index.isin(rna_ibm_embargo)]
print(len(tpm_transcripts))
tpm_transcripts.to_csv('temp/dmc_'+release+'_transcripts_tpm')

1415
1357
1415
1357
1415
1357
1415
1357


In [25]:
prev_counts_genes = tc.get(name='depmap-rnaseq-expression-data-80ef', version=14, file='dmc_'+prevname+'_counts')
print('shoud be None')
print(set(prev_counts_genes.index) - set(counts_genes.index))
print("new lines")
newlines = set(counts_genes.index) - set(prev_counts_genes.index) 
newlines

[##################]100% |  16.8 MiB/s | 611.0 MiB / 611.0 MiB | Time:  0:00:36


shoud be None
set()
new lines


set()

In [ ]:
tc.update_dataset(dataset_permaname="depmap-rnaseq-expression-data-80ef",
                 upload_file_path_dict={'temp/dmc_'+release+'_counts': 'NumericMatrixCSV',
                                       'temp/dmc_'+release+'_tpm': 'NumericMatrixCSV',
                                       'temp/dmc_'+release+'_proteincoding_tpm': 'NumericMatrixCSV',
                                       'temp/dmc_'+release+'_transcripts_tpm': 'NumericMatrixCSV'},
                  dataset_description=
"""
# DMC RNA

* Version 1-3 DMC 19Q1*

version 3 contains the correct data for 19Q1

version 2 contains correct TPM transcript data (in version 1 transcript data contains only gene level not transcript level data)

* Version 4 DMC 19Q2*

* Version 5 DMC 19Q3*

* Version 6 DMC 19Q4*

Adding 93 new cell lines - Blacklisted - IBM
Some cells lines have been removed because they:

 - had too many 0 values = ["ACH-001388" "ACH-001577" "ACH-001767" "ACH-002463"] 

Some cells lines have been flagged as:

 - None

* Version 7 DMC 19Q4

removing blacklisted

* Version 8 DMC 19Q4

removing blacklisted in transcripts

* Version 9 DMC 19Q4

uploading as numeric matrix

* Version 10 DMC 20Q1
unknown reupload

* Version 11 DMC 20Q2
adding  new cell lines

* Version 12 DMC 20Q2
unknown reupload

* Version 13 DMC 20Q2
removing some missed blacklisted lines

* Version 14 DMC 20Q2
Adding one missing line

* Version 15 DMC 20Q3
nothing different from  20Q2. no new cell lines added

* Version 16 DMC 20Q3
Some wrong annotations in the blacklists

* Version 17 DMC 20Q3
Updated annotations in the blacklists


data is aligned to hg38

read count data is created using RSEM, which, when a read maps to multiple places, splits the counts between genes, with the weight based on the likelihood that it came from one gene or the other, so counts data may not be integers

TPM data is log2-transformed with a pseudo count of 1 added.

Reads/transcript files unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are DepMap cell line IDs

Columns: In the complete TPM and read counts datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding TPM dataset column names are HGNC_symbol (Entrez_ID). In the TPM transcript dataset column names are HGNC_symbol (Transcript_ID) - the HGNC symbols are not unique, use the transcript IDs for unique identifiers.

NEW LINES:
"""+newlines)

In [ ]:
AddToVirtual(virtual_dmc, "depmap-rnaseq-expression-data-80ef", files=[('CCLE_RNAseq_reads', 'dmc_'+release+'_counts'),('CCLE_expression_full', 'dmc_'+release+'_tpm'),('CCLE_expression', 'dmc_'+release+'_proteincoding_tpm'),('CCLE_RNAseq_transcripts', 'dmc_'+release+'_transcripts_tpm')])

### public

In [26]:
prevprevcells = set(tc.get(name='depmap-rnaseq-expression-data-363a', file='internal_'+prevprevname+'_proteincoding_tpm',version=prevprevversion).index.tolist())

In [84]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(counts_genes))
counts_genes = counts_genes[counts_genes.index.isin(prevprevcells)]
counts_genes = counts_genes[~counts_genes.index.isin(rna_dmc_embargo)]
print(len(counts_genes))
counts_genes.to_csv('temp/public_'+release+'_counts')
print(len(tpm_genes))
tpm_genes = tpm_genes[tpm_genes.index.isin(prevprevcells)]
tpm_genes = tpm_genes[~tpm_genes.index.isin(rna_dmc_embargo)]
print(len(tpm_genes))
tpm_genes.to_csv('temp/public_'+release+'_tpm')
print(len(tpm_proteincoding))
tpm_proteincoding = tpm_proteincoding[tpm_proteincoding.index.isin(prevprevcells)]
tpm_proteincoding = tpm_proteincoding[~tpm_proteincoding.index.isin(rna_dmc_embargo)]
print(len(tpm_proteincoding))
tpm_proteincoding.to_csv('temp/public_'+release+'_proteincoding_tpm')
print(len(tpm_transcripts))
tpm_transcripts = tpm_transcripts[tpm_transcripts.index.isin(prevprevcells)]
tpm_transcripts = tpm_transcripts[~tpm_transcripts.index.isin(rna_dmc_embargo)]
print(len(tpm_transcripts))
tpm_transcripts.to_csv('temp/public_'+release+'_transcripts_tpm')

1311
1311
1357
1311
1357
1311
1357
1311


In [85]:
prev_counts_genes = tc.get(name='depmap-rnaseq-expression-data-ccd0', version=21, file='public_'+prevname+'_counts')
print('shoud be None')
print(set(prev_counts_genes.index) - set(counts_genes.index))
print("new lines")
newlines = set(counts_genes.index) - set(prev_counts_genes.index) 
newlines

shoud be None
set()
new lines


{'ACH-001533',
 'ACH-001574',
 'ACH-001625',
 'ACH-001632',
 'ACH-001847',
 'ACH-002014'}

In [86]:
tc.update_dataset(dataset_permaname="depmap-rnaseq-expression-data-ccd0",
                 upload_file_path_dict={'temp/public_'+release+'_counts': 'NumericMatrixCSV',
                                       'temp/public_'+release+'_tpm': 'NumericMatrixCSV',
                                       'temp/public_'+release+'_proteincoding_tpm': 'NumericMatrixCSV',
                                       'temp/public_'+release+'_transcripts_tpm': 'NumericMatrixCSV'},
                  dataset_description=
"""
# PUBLIC RNA

* Version 1-2 Public 18Q1*

Original source (`CCLE_DepMap_18Q1_RNAseq_reads_20180214.gct`, `CCLE_DepMap_18Q1_RNAseq_RPKM_20180214.gct`) downloaded from portals.broadinstitute.org/ccle
RPKM file is log2(RPKM) with a "noisy floor" around -3 (-3 + N(0, 0.1))

* Version 3-5 Public 18Q2*

gene expression data (RNAseq for1,076 cell lines, including data for 28 newly released cell lines)

original source: (`/xchip/ccle_dist/public/DepMap_18Q2/CCLE_DepMap_18Q2_RNAseq_RPKM_20180502.gct`, `/xchip/ccle_dist/public/DepMap_18Q2/CCLE_DepMap_18Q2_RNAseq_reads_20180502.gct`)
* Version 6-7 Public 18Q3*

gene expression data (80 newly released cell lines)

Original data source:

`/xchip/ccle_dist/public/DepMap_18Q3/CCLE_DepMap_18q3_RNAseq_reads_20180718.gct`
`/xchip/ccle_dist/public/DepMap_18Q3/CCLE_DepMap_18q3_RNAseq_RPKM_20180718.gct`

RPKM data are log2-transformed with a noisy floor at -3 (-3 + N(0, 0.1)). Reads file unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are Broad (Arxspan) cell line IDs. Mapping between Broad IDs and CCLE IDs can be done using a R or python package

To install R implementation: options(repos = c("https://iwww.broadinstitute.org/~datasci/R-packages", "https://cran.cnr.berkeley.edu")) install.packages('celllinemapr')

To install python implementation: pip install https://intranet.broadinstitute.org/~datasci/python-packages/cell_line_mapper-0.1.9.tar.gz)

Columns: In the complete RPKM and read datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding RPKM and read datasets column names are HGNC_symbol (Entrez_ID)

version 7 removes duplicate gene names from the protein coding datasets

* Version 8-9 Public 18Q4*

_ 18Q4 transcript level data is found in version 11. (In versions 8-9 transcript data contains only gene level not transcript level data)

changing to TPM expression

Original data source:

`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_DepMap_18Q4_rsem_genes_tpm_20181029.txt`
`/xchip/ccle_dist/broad_only/CMAG/expression/CCLE_DepMap_18Q4_rsem_transcripts_tpm_20181029.gct`
`/xchip/ccle_dist/public/DepMap_18Q4/CCLE_DepMap_18q4_RNAseq_reads_20181029.gct`

`/xchip/ccle_dist/public/DepMap_18Q4/CCLE_DepMap_18q4_RNAseq_RPKM_20181029.gct`

TPM data is subsetted to just public cell lines using the cell line found in the RPKM dataset.

TPM data is the primary expression data now. It is log2-transformed with a pseudo count of 1 added

RPKM data are log2-transformed with a pseudo count of 1 added. RPKM values are no longer thresholded.

Reads/transcript files unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are DepMap (Arxspan) cell line IDs

Columns: In the complete TPM, TPM transcripts, RPKM and read datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding TPM and read datasets column names are HGNC_symbol (Entrez_ID)

* Version 10-12 Public 19Q1*

version 12 contains the correct data for 19Q1

version 11 contains the correct transcript level data for 19Q1 and 18Q4

* Version 13 Public 19Q2*

* Version 14 Public 19Q3*

* Version 15 Public 19Q4*
Adding 93 new cell lines - Blacklisted - IBM - DMC
Some cells lines have been removed because they:

 - had too many 0 values = ["ACH-001388" "ACH-001577" "ACH-001767" "ACH-002463"] 

Some cells lines have been flagged as:

 - None
 
* Version 16 Public 19Q4*
removing unauthorized cell lines

* Version 17 Public 20Q1*
adding 6 new lines

* Version 18 Public 20Q1
Unknown reupdate

* Version 19 Public 20Q2
adding  new cell lines

* Version 20 Public 20Q2
unknown reupload

* Version 21 Public 20Q2
removing some missed blacklisted lines

* Version 22 Public 20Q3
nothing different from  20Q2. no new cell lines added

* Version 23 Public 20Q3
Some wrong annotations in the blacklists

* Version 24 Public 20Q3
Updated annotations in the blacklists

* Version 25 Public 20Q3
Updated the dmc list

* Version 25 Public 20Q3
Readding a cell line that was already in public before.

data is hg38 aligned

read count data is created using RSEM, which, when a read maps to multiple places, splits the counts between genes, with the weight based on the likelihood that it came from one gene or the other, so counts data may not be integers

TPM data is log2-transformed with a pseudo count of 1 added. log2(X+1)

Reads/transcript files unaltered aside from formatting row / column names.

The ProteinCoding datasets contain just protein coding genes and are based on protein coding annotations from https://www.genenames.org/cgi-bin/download (locus_group == "protein-coding gene")

Rows: are DepMap cell line IDs

Columns: In the complete TPM and read counts datasets column names are HGNC_symbol (Ensembl_ID), while in the ProteinCoding TPM dataset column names are HGNC_symbol (Entrez_ID). In the TPM transcript dataset column names are HGNC_symbol (Transcript_ID) - the HGNC symbols are not unique, use the transcript IDs for unique identifiers.

NEW LINES:
"""+str(newlines))

Uploading public_20Q3_counts...
hitting https://cds.team/taiga/api/datafile/f22029659d154cc2b193cb6b219ca211
Conversion and upload...:
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Scanning through file to determine size (line 1001)
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 250
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion 

	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 500
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in progress, line 750
	 Conversion in prog

	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploadin

'8cb5f6cdd90542789bfbe897585b9a03'

In [87]:
AddToVirtual(virtual_public, "depmap-rnaseq-expression-data-ccd0", files=[('CCLE_RNAseq_reads', 'public_'+release+'_counts'),('CCLE_expression_full', 'public_'+release+'_tpm'),('CCLE_expression', 'public_'+release+'_proteincoding_tpm'),('CCLE_RNAseq_transcripts', 'public_'+release+'_transcripts_tpm')])

[('CCLE_RNAseq_reads', 'depmap-rnaseq-expression-data-ccd0.26/public_20Q3_counts'), ('CCLE_expression_full', 'depmap-rnaseq-expression-data-ccd0.26/public_20Q3_tpm'), ('CCLE_expression', 'depmap-rnaseq-expression-data-ccd0.26/public_20Q3_proteincoding_tpm'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-ccd0.26/public_20Q3_transcripts_tpm'), ('Achilles_common_essentials', 'avana-public-tentative-20q3-3e73.6/pan_dependent_genes'), ('Achilles_gene_dependency', 'avana-public-tentative-20q3-3e73.6/gene_dependency'), ('Achilles_raw_readcounts_failures', 'avana-public-tentative-20q3-3e73.6/raw_readcounts_failing'), ('CCLE_fusions_unfiltered', 'gene-fusions-6212.14/unfiltered_fusions_20Q3'), ('Achilles_guide_efficacy', 'avana-public-tentative-20q3-3e73.6/guide_efficacy'), ('Achilles_gene_effect_unscaled', 'avana-public-tentative-20q3-3e73.6/gene_effect_unscaled'), ('Achilles_high_variance_genes', 'avana-public-tentative-20q3-3e73.6/high_variance_genes'), ('CCLE_fusions', 'gene-fu

KeyboardInterrupt: 

## Fusion post processing

In [ ]:
renamings

In [ ]:
terra.waitForSubmission(refworkspace, submission_id2)

In [ ]:
submission_id2 = refwm.create_submission("Aggregate_Fusion_Calls", 'All_samples')
terra.waitForSubmission(refworkspace, submission_id2)

In [ ]:
aggregated = refwm.get_sample_sets().loc['All_samples']['fusions_star']

In [ ]:
! gsutil cp $aggregated "temp/expression.fusion.tsv"

In [ ]:
file = pd.read_csv('temp/expression.fusion.tsv', header=0, sep='\t', quotechar='"', error_bad_lines=False, index_col=None)
file["DepMap_ID"] = [i.split('.')[0] for i in file['DepMap_ID']]

In [ ]:
print(len(file))
renaming = removeOlderVersions(names=set(file['DepMap_ID']), refsamples=refwm.get_samples(), arxspan_id="arxspan_id", version="version")
file = file[file['DepMap_ID'].isin(renaming.keys())].replace({'DepMap_ID':renaming}).reset_index(drop=True)
print(len(file))
print(file['DepMap_ID'][:10])
file.to_csv('temp/expression.fusion.tsv', sep='\t')

### Overview

This document contains the code used to generate the unfiltered and filtered versions of the fusion datasets for the release. The bottom of the document also contains some comparisons between the release fusion dataset, CCLE2 fusion calls, and the translocation data from CCLE2.

In [ ]:
%%R
source('src/load_libraries_and_annotations.R')
source("src/CCLE_postp_function.R")
library('cdsomics')

## Generate filtered fusion table

Release: `r release`

We want to apply filters to the fusion table to reduce the number of artifacts in the dataset. Specifically, we filter the following:

* Remove fusions involving mitochondrial chromosomes, or HLA genes, or immunoglobulin genes
* Remove red herring fusions (from STAR-Fusion annotations column)
* Remove recurrent in CCLE (>= 25 samples)
* Remove fusion with (SpliceType=" INCL_NON_REF_SPLICE" and LargeAnchorSupport="No" and FFPM < 0.1)
* Remove fusions with FFPM < 0.05 (STAR-Fusion suggests using 0.1, but looking at the translocation data, this looks like it might be too aggressive)

with the functions:
- readFusions
- filterFusions
- prepare_depmap_fusion_data_for_taiga

In [ ]:
%%R
unfiltered_fusions <- readFusions('temp/expression.fusion.tsv')
filtered_fusions <- filterFusions(unfiltered_fusions)

filtered_fusions <- prepare_depmap_fusion_data_for_taiga(filtered_fusions)
unfiltered_fusions <- prepare_depmap_fusion_data_for_taiga(unfiltered_fusions)

filtered_fusions <- filtered_fusions[,2:ncol(filtered_fusions)]
unfiltered_fusions <- unfiltered_fusions[,2:ncol(unfiltered_fusions)]

In [ ]:
%%R
# Save the files (to be uploaded to taiga)
write.table(
  unfiltered_fusions,
  file = paste0('temp/fusions_',release, '_unfiltered.all'),
  sep = ',', quote = F, row.names = F
)
write.table(
  filtered_fusions,
  file = paste0('temp/fusions_', release, '_filtered.all'),
  sep = ',', quote = F, row.names = F
)

## Uploading to Taiga

In [ ]:
tc.update_dataset(dataset_permaname="depmap-fusions-7990",
                     upload_file_path_dict={'temp/fusions_'+release+'_filtered.all': 'TableCSV',
                                        'temp/fusions_'+release+'_unfiltered.all': 'TableCSV'},
                 dataset_description="""
# Fusions

filtered and unfiltered fusion files from Broad RNAseq data mapped to hg38

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

version 1-4: guillaume releases
version 5: 19Q3 release
version 6:  adding missing samples in Terra merge files

## ** version 7 Internal 19Q4****
Adding 17 new cell lines, 3 reprioritized cell lines. log2(COPY RATIO+1). 
Some cells lines have been flagged as:

## ** version 8 Internal 20Q1****
Adding 6 new lines

## ** version 9 Internal 20Q2****
Adding 50 new lines

## ** version 10 Internal 20Q3****
nothing different from  20Q2. no new cell lines added


Description: Gene fusions derived from RNAseq data.

Rows: cell lines, IDs contained in the column DepMap_ID
LeftGene and RightGene separated by an ampersand ("&").

Unfiltered data contains all output fusions, while the filtered data uses the filters suggested by the star fusion docs. These filters are:
- FFPM > 0.1 -  a cutoff of 0.1 means&nbsp;at least 1 fusion-supporting RNAseq fragment per 10M total reads
- Remove known false positives, such as GTEx recurrent fusions and certain paralogs
- Genes that are next to each other
- Fusions with mitochondrial breakpoints
- Removing fusion involving mitochondrial chromosomes or HLA genes
- Removed common false positive fusions (red herring annotations as described in the STAR-Fusion docs)
- Recurrent fusions observed in CCLE across cell lines (in 25 or more samples)
- Removed fusions where SpliceType='INCL_NON_REF_SPLICE' and LargeAnchorSupport='NO_LDAS' and FFPM < 0.1
- FFPM < 0.05
""")

In [34]:
unfiltered = pd.read_csv('temp/fusions.'+release+'.unfiltered.all', sep='\t')#,index_col=None)
filtered = pd.read_csv('temp/fusions.'+ release+'.filtered.all', sep='\t')#,index_col=None)

FileNotFoundError: [Errno 2] File b'temp/fusions_20Q3_unfiltered' does not exist: b'temp/fusions_20Q3_unfiltered'

### Internal

In [ ]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(filtered))
filtered = filtered[~filtered.DepMap_ID.isin(blacklist)]
print(len(filtered))
filtered.to_csv('temp/filtered_fusions_'+release, index=False)
print(len(unfiltered))
unfiltered= unfiltered[~unfiltered.DepMap_ID.isin(blacklist)]
print(len(unfiltered))
unfiltered.to_csv('temp/unfiltered_fusions_'+release, index=False)

In [ ]:
prev_unfilt = tc.get(name='gene-fusions-8b7a', version=14, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prev_unfilt.DepMap_ID) - set(unfiltered.DepMap_ID))
print("new lines")
newlines = set(unfiltered.DepMap_ID) - set(prev_unfilt.DepMap_ID) 
newlines

In [ ]:
tc.update_dataset(dataset_permaname="gene-fusions-8b7a",
                 upload_file_path_dict={'temp/filtered_fusions_'+release: 'TableCSV',
                                       'temp/unfiltered_fusions_'+release: 'TableCSV'},
                  dataset_description=
"""
# Internal Fusions

Description: Gene fusions derived from RNAseq data.

Rows: cell lines

Original Raw Data: Generated by Mahmoud Ghandi on April 25, 2017. Can be found at xchip_ccle_dist/broad_only/unpublished_Novartis_data/RNAseq/fusions.txt

Version 3: added a column containing the Broad_ID

* Version 4-5 Internal 19Q1*

version 5 contains the correct data for 19Q1

Description: Gene fusions derived from RNAseq data.

Rows: cell lines

* Version 6 Internal 19Q2*

* Version 7*

Josh D added "common_fusion_matrix".

Binary matrix of the most common gene fusions (those where the two involved genes are fused in at least 5 cell lines) with no additional filtering. Use at your own risk.

Description: Gene fusions derived from RNAseq data.

Rows: cell lines

* Version 8 Internal 19Q3*

* Version 9 Internal 19Q4*

* Version 10 Internal 20Q1*
Adding new lines

* Version 11 Internal 20Q1*
Adding a file

* Version 12 Internal 20Q2*
Adding 50 new lines

* Version 13 Internal 20Q3*
nothing different from  20Q2. no new cell lines added

* Version 14 Internal 20Q3*
issues with the blacklists

Description: Gene fusions derived from RNAseq data.

Rows: cell lines, IDs contained in the column DepMap_ID
LeftGene and RightGene separated by an ampersand ("&").

Unfiltered data contains all output fusions, while the filtered data uses the filters suggested by the star fusion docs. These filters are:
- FFPM > 0.1 -  a cutoff of 0.1 means&nbsp;at least 1 fusion-supporting RNAseq fragment per 10M total reads
- Remove known false positives, such as GTEx recurrent fusions and certain paralogs
- Genes that are next to each other
- Fusions with mitochondrial breakpoints
- Removing fusion involving mitochondrial chromosomes or HLA genes
- Removed common false positive fusions (red herring annotations as described in the STAR-Fusion docs)
- Recurrent fusions observed in CCLE across cell lines (in 25 or more samples)
- Removed fusions where SpliceType='INCL_NON_REF_SPLICE' and LargeAnchorSupport='NO_LDAS' and FFPM < 0.1
- FFPM < 0.05

NEW LINES:
"""+newlines)

AddToVirtual('depmap-a0ab', "gene-fusions-8b7a", files=[('CCLE_fusions_unfiltered', 'unfiltered_fusions_'+release),('CCLE_fusions', 'filtered_fusions_'+release)])

AddToVirtual(virtual_internal, "gene-fusions-8b7a", files=[('CCLE_fusions_unfiltered', 'unfiltered_fusions_'+release),('CCLE_fusions', 'filtered_fusions_'+release)])

### IBM

same as internal

### DMC

In [37]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(filtered))
filtered = filtered[~filtered.DepMap_ID.isin(rna_ibm_embargo)]
print(len(filtered))
filtered.to_csv('temp/filtered_fusions_'+release, index=False)

print(len(unfiltered))
unfiltered  = unfiltered[~unfiltered.DepMap_ID.isin(rna_ibm_embargo)]
unfiltered.to_csv('temp/unfiltered_fusions_'+release, index=False)
print(len(unfiltered))

35478
34623
358959
351412


In [39]:
prev_unfilt = tc.get(name='gene-fusions-375f', version=7, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prev_unfilt.DepMap_ID) - set(unfiltered.DepMap_ID))
print("new lines")
newlines = set(unfiltered.DepMap_ID) - set(prev_unfilt.DepMap_ID) 
newlines

[##################]100% |  46.2 MiB/s |  79.8 MiB /  79.8 MiB | Time:  0:00:01


shoud be None
{'ACH-001708', 'ACH-001690', 'ACH-001519', 'ACH-002038', 'ACH-001512', 'ACH-001398', 'ACH-001441', 'ACH-001419', 'ACH-001649', 'ACH-001528', 'ACH-001516', 'ACH-001384', 'ACH-001388', 'ACH-001991', 'ACH-001395', 'ACH-001389', 'ACH-001393', 'ACH-001709', 'ACH-001569', 'ACH-001541', 'ACH-002039', 'ACH-001688', 'ACH-001422', 'ACH-002022', 'ACH-002067', 'ACH-001370', 'ACH-002026', 'ACH-001679', 'ACH-001573', 'ACH-002023', 'ACH-001694', 'ACH-001339'}
new lines


set()

In [41]:
tc.update_dataset(dataset_permaname="gene-fusions-375f",
                 upload_file_path_dict={'temp/filtered_fusions_'+release: 'TableCSV',
                                       'temp/unfiltered_fusions_'+release: 'TableCSV'},
                  dataset_description=
"""
# DMC Fusions

* Version 1-2 DMC 19Q1*

version 2 contains the correct data for 19Q1

Description: Gene fusions derived from RNAseq data.

Rows: cell lines

Unfiltered data contains all output fusions, while the filtered data uses the filters suggested by the star fusion docs. These filters are:
- FFPM > 0.1 -  a cutoff of 0.1 means&nbsp;at least 1 fusion-supporting RNAseq fragment per 10M total reads
- Remove known false positives, such as GTEx recurrent fusions and certain paralogs
- Genes that are next to each other
- Fusions with mitochondrial breakpoints

* Version 3 DMC 19Q2*

* Version 4 DMC 19Q3*

* Version 5 DMC 19Q4*

* Version 6 DMC 20Q1*

* Version 7 DMC 20Q2*
adding 50 new lines

* Version 7 DMC 20Q2*
Unknown change

* Version 9 DMC 20Q3*
nothing different from  20Q2. no new cell lines added

* Version 10 DMC 20Q3*
updating the blacklists

* Version 11 DMC 20Q3*
issues with the blacklists resolved
Description: Gene fusions derived from RNAseq data.

* Version 12 DMC 20Q3*
update in dmc list

Rows: cell lines, IDs contained in the column DepMap_ID
LeftGene and RightGene separated by an ampersand ("&").

Unfiltered data contains all output fusions, while the filtered data uses the filters suggested by the star fusion docs. These filters are:
- FFPM > 0.1 -  a cutoff of 0.1 means&nbsp;at least 1 fusion-supporting RNAseq fragment per 10M total reads
- Remove known false positives, such as GTEx recurrent fusions and certain paralogs
- Genes that are next to each other
- Fusions with mitochondrial breakpoints
- Removing fusion involving mitochondrial chromosomes or HLA genes
- Removed common false positive fusions (red herring annotations as described in the STAR-Fusion docs)
- Recurrent fusions observed in CCLE across cell lines (in 25 or more samples)
- Removed fusions where SpliceType='INCL_NON_REF_SPLICE' and LargeAnchorSupport='NO_LDAS' and FFPM < 0.1
- FFPM < 0.05
""")

Uploading filtered_fusions_20Q3...
hitting https://cds.team/taiga/api/datafile/ec7646366a644555a04dce2d4c5512e9
Conversion and upload...:
	 Uploading to S3

	 Done: filtered_fusions_20Q3 properly converted and uploaded
Uploading unfiltered_fusions_20Q3...
hitting https://cds.team/taiga/api/datafile/ec7646366a644555a04dce2d4c5512e9
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: unfiltered_fusions_20Q3 properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 29e43889e98b4e76afaeb3bfd0e4b6d1 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/29e43889e98b4e76afaeb3bfd0e4b6d1


'29e43889e98b4e76afaeb3bfd0e4b6d1'

In [42]:
AddToVirtual(virtual_dmc, "gene-fusions-375f", files=[('CCLE_fusions', 'filtered_fusions_'+release),('CCLE_fusions_unfiltered', 'unfiltered_fusions_'+release)])

[('CCLE_fusions', 'gene-fusions-375f.12/filtered_fusions_20Q3'), ('CCLE_fusions_unfiltered', 'gene-fusions-375f.12/unfiltered_fusions_20Q3'), ('Achilles_gene_effect', 'avana-consortium-20q3-902b.1/gene_effect'), ('CCLE_segment_cn', 'depmap-cn-data-9b9d.15/dmc_20Q3_segs_cn'), ('sample_info', 'dmc-20q3-033d.11/sample_info'), ('Achilles_common_essentials', 'avana-consortium-20q3-902b.1/pan_dependent_genes'), ('CCLE_RNAseq_transcripts', 'depmap-rnaseq-expression-data-80ef.17/dmc_20Q3_transcripts_tpm'), ('CCLE_expression', 'depmap-rnaseq-expression-data-80ef.17/dmc_20Q3_proteincoding_tpm'), ('common_essentials', 'avana-consortium-20q3-902b.1/essential_genes'), ('nonessentials', 'avana-consortium-20q3-902b.1/nonessential_genes'), ('Achilles_replicate_map', 'avana-consortium-20q3-902b.1/replicate_map'), ('Achilles_guide_map', 'avana-consortium-20q3-902b.1/guide_gene_map'), ('Achilles_guide_efficacy', 'avana-consortium-20q3-902b.1/guide_efficacy'), ('CCLE_mutations', 'depmap-mutation-calls-dfc

KeyboardInterrupt: 

In [ ]:
set(unfiltered.DepMap_ID) - set(counts_genes.index)

### Public

In [90]:
## removing first blacklisted, then embargoed, to create two datasets
print(len(filtered))
filtered = filtered[filtered.DepMap_ID.isin(prevprevcells)]
filtered = filtered[~filtered.DepMap_ID.isin(rna_dmc_embargo)]
print(len(filtered))
filtered.to_csv('temp/filtered_fusions_'+release, index=False)

print(len(unfiltered))
unfiltered = unfiltered[unfiltered.DepMap_ID.isin(prevprevcells)]
unfiltered = unfiltered[~unfiltered.DepMap_ID.isin(rna_dmc_embargo)]
print(len(unfiltered))
unfiltered.to_csv('temp/unfiltered_fusions_'+release, index=False)

34623
33688
351412
344846


In [91]:
prev_unfilt = tc.get(name='gene-fusions-6212', version=10, file='unfiltered_fusions_'+prevname)
print('shoud be None')
print(set(prev_unfilt.DepMap_ID) - set(unfiltered.DepMap_ID))
print("new lines")
newlines = set(unfiltered.DepMap_ID) - set(prev_unfilt.DepMap_ID) 
newlines

shoud be None
{'ACH-001690', 'ACH-002038', 'ACH-001398', 'ACH-001441', 'ACH-001419', 'ACH-001649', 'ACH-001528', 'ACH-001516', 'ACH-001991', 'ACH-001395', 'ACH-001389', 'ACH-001709', 'ACH-001569', 'ACH-001541', 'ACH-002039', 'ACH-001688', 'ACH-001422', 'ACH-002022', 'ACH-001370', 'ACH-002026', 'ACH-001573', 'ACH-002023', 'ACH-001694', 'ACH-001339'}
new lines


{'ACH-001533',
 'ACH-001574',
 'ACH-001625',
 'ACH-001632',
 'ACH-001847',
 'ACH-002014'}

In [92]:
tc.update_dataset(dataset_permaname="gene-fusions-6212",
                 upload_file_path_dict={'temp/filtered_fusions_'+release: 'TableCSV',
                                       'temp/unfiltered_fusions_'+release: 'TableCSV'},
                  dataset_description=
"""
# PUBLIC Fusions

* Version 1 Public 2017 data*

Description: Gene fusions derived from RNAseq data.

Rows: cell lines, ID contained in the column Broad_ID

Original Raw Data: Generated by Mahmoud Ghandi on April 25, 2017. Can be found at xchip_ccle_dist/broad_only/unpublished_Novartis_data/RNAseq/fusions.txt

* Version 2-3 Public 19Q1*

version 3 contains the correct data for 19Q1

* Version 4-6 Public 19Q2*

in version 5 formatting of the columns is improved

* Version 7 Public 19Q3*

* Version 8 Public 19Q4*

* Version 9 Public 20Q1*

* Version 10 Public 20Q2*
adding 50 new lines

* Version 11 Public 20Q3*
nothing different from  20Q2. no new cell lines added

* Version 12 Public 20Q3*
updated blacklists

* Version 13 Public 20Q3*
issues with the blacklists

* Version 14 Public 20Q3*
updating the dmc list 

* Version 15 Public 20Q3*
re adding two missing, already released samples

Description: Gene fusions derived from RNAseq data.

Rows: cell lines, IDs contained in the column DepMap_ID
LeftGene and RightGene separated by an ampersand ("&").

Unfiltered data contains all output fusions, while the filtered data uses the filters suggested by the star fusion docs. These filters are:
- FFPM > 0.1 -  a cutoff of 0.1 means&nbsp;at least 1 fusion-supporting RNAseq fragment per 10M total reads
- Remove known false positives, such as GTEx recurrent fusions and certain paralogs
- Genes that are next to each other
- Fusions with mitochondrial breakpoints
- Removing fusion involving mitochondrial chromosomes or HLA genes
- Removed common false positive fusions (red herring annotations as described in the STAR-Fusion docs)
- Recurrent fusions observed in CCLE across cell lines (in 25 or more samples)
- Removed fusions where SpliceType='INCL_NON_REF_SPLICE' and LargeAnchorSupport='NO_LDAS' and FFPM < 0.1
- FFPM < 0.05

""")
AddToVirtual(virtual_public, "gene-fusions-6212", files=[('CCLE_fusions', 'filtered_fusions_'+release),('CCLE_fusions_unfiltered', 'unfiltered_fusions_'+release)])

Uploading filtered_fusions_20Q3...
hitting https://cds.team/taiga/api/datafile/4736470152754737b3e76db7ef562b75
Conversion and upload...:
	 Uploading to S3

	 Done: filtered_fusions_20Q3 properly converted and uploaded
Uploading unfiltered_fusions_20Q3...
hitting https://cds.team/taiga/api/datafile/4736470152754737b3e76db7ef562b75
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: unfiltered_fusions_20Q3 properly converted and uploaded
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 60b366e5d24040898a5d5e3b0e43d726 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/60b366e5d24040898a5d5e3b0e43d726
[('CCLE_fusions', 'gene-fusions-6212.15/filtered_fusions_20Q3'), ('CCLE_fusions_unfiltered', 'gene-fusions-6212.15/unfiltered_fusions_20Q3'), ('

## [OPTIONAL] IF want to merge here instead of on Terra:

In [ ]:
newsamples

In [ ]:
prevsamplesets = ['CCLE_19Q3interim',samplesetname]
samples = []
for i in prevsamplesets:
    samples.extend(refwm.get_sample_sets().loc[i].samples)
res = []
terrasamp = refwm.get_samples()
for i, sample in enumerate(samples):
    res.append(terrasamp.loc[sample])
    genes_fusion = res[i]['fusion_predictions_abridged']
    rsem_genes_transcripts = res[i]['rsem_isoforms']
    rsem_genes_expected_count = res[i]['rsem_genes']
    ! gsutil cp $rsem_genes_expected_count 'temp/' && gsutil cp $rsem_genes_transcripts 'temp/' && gsutil cp $genes_fusion 'temp/'

In [ ]:
mainres = refwm.get_sample_sets().loc['DM19Q2_PATHS_CORRECTED_V2']
maingenes_fusion = mainres['fusions_star']
mainrsem_genes_tpm = mainres['rsem_genes_tpm']
mainrsem_genes_transcripts = mainres['rsem_transcripts_tpm']
mainrsem_genes_expected_count = mainres['rsem_genes_expected_count']
! gsutil cp $mainrsem_genes_expected_count "temp/" && gsutil cp $mainrsem_genes_transcripts "temp/" && gsutil cp $maingenes_fusion "temp/expression.fusion.tsv" && gsutil cp $mainrsem_genes_tpm "temp/"

In [ ]:
mainres['rsem_genes_expected_count']

In [ ]:
addSamplesRSEMToMain(res,mainres)

In [ ]:
genes_fusion = ['temp/'+val['fusion_predictions_abridged'].split('/')[-1] for val in res]
addToMainFusion(genes_fusion,'temp/expression.fusion.tsv')